<a href="https://colab.research.google.com/github/Mamadyc/D-fi-2---classification-de-textes/blob/main/Rapport_de_stage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#############################################################
#################### Analyse Spatiale 67-68 #################
#############################################################
############### Fusion des bases ############################
#### base 1988-2019 (67-68)
## Dep 68
library(readr)
KAlsace_1988_2019_R68 <- read_delim("KAlsace_1988-2019_R68.csv",
                                    delim = ";", escape_double = FALSE, trim_ws = TRUE)
View(KAlsace_1988_2019_R68)

## Dep 67
library(readr)
R67_ALSACE <- read_csv("R67_ALSACE.csv")
View(R67_ALSACE)


# Supprimer la notation scientifique et forcer le format 9 chiffres avec les IRIS

KAlsace_1988_2019_R68$IRIS <- sprintf("%09s", KAlsace_1988_2019_R68$IRIS)

R67_ALSACE$IRIS <- sprintf("%09s", R67_ALSACE$IRIS )

table(nchar(KAlsace_1988_2019_R68$IRIS))
table(nchar(R67_ALSACE$IRIS))

R67_ALSACE$T_HISTO <- as.double(R67_ALSACE$T_HISTO)

R67_ALSACE$IRIS <- as.numeric(R67_ALSACE$IRIS)

KAlsace_1988_2019_R68$IRIS <- as.numeric(KAlsace_1988_2019_R68$IRIS)

### Fusion des bases 67-68
library(dplyr)
Cancers_Alsace <- bind_rows(R67_ALSACE,KAlsace_1988_2019_R68)

Cancers_Alsace$IRIS <- sprintf("%09s", Cancers_Alsace$IRIS)
table(nchar(Cancers_Alsace$IRIS))

Cancers_Alsace$T_INCIDENCE_DATE <- substr(Cancers_Alsace$T_INCIDENCE_DATE,1,4)  ## maintenir seulement l'ann?e d'incidence et non les mois et les jours

################################################################################
Cancers_Alsace        ### C'est nos deux bases fusionn?e             ###########
################################################################################
######## Fonction de nettoyage centr?e sur la colonne 'IRIS' pour avoir uniquement 2006-2019 ####################

table(nchar(Cancers_Alsace$IRIS))
library(dplyr)

nettoyer_donnees_iris <- function(df, an_debut = 2010, an_fin = 2017) {
  df_clean <- df %>%
    filter(T_INCIDENCE_DATE >= an_debut, T_INCIDENCE_DATE <= an_fin) %>%  # on va Garder les Ann?es souhait?es 2006-2019
    filter(!is.na(IRIS))  # Supprimer les lignes avec iris manquant

  return(df_clean)
}

################# Application ? la base compl?te##################################
################# Fonction ############# Base des donn?es #########################
Geo <- nettoyer_donnees_iris(Cancers_Alsace) ##
###################################################################################
#.................................................................................................................................
## les donn?es de la base R67_ALSACE contient des iris manquants pour la p?riode de 2006-2009
## et certains informations sur les quintiles manquent aussi donc je vais travailler d'abord uniquement sur les iris pr?sent
## et apr?s adapter mon code en fonction des suggestions.
#.................................................................................................................................

## G?o d?vient notre nouvelle base avec uniquement les cas pr?sent de 2006-2019 ....

sapply(R67_ALSACE, function(x) sum(is.na(x)))
sapply(KAlsace_1988_2019_R68, function(x) sum(is.na(x)))
sapply(Geo, function(x) sum(is.na(x)))
sapply(Cancers_Alsace, function(x) sum(is.na(x)))

Geo$IRIS <- as.numeric(Geo$IRIS)

summary(Geo)
str(G?o)
table(nchar(Geo$IRIS))
table(nchar(Geo))

write.csv(Geo, "Geo.csv", row.names = FALSE)
write.csv(Cancers_Alsace, "Cancers_Alsace.csv", row.names = FALSE)
write.csv(R67_ALSACE, "R67_ALSACE.csv", row.names = FALSE)
write.csv(KAlsace_1988_2019_R68, "KAlsace_1988_2019_R68.csv", row.names = FALSE)
Geo <- sprintf("%09s", Geo$IRIS)

write.csv(G?o, "G?o.csv", row.names = FALSE)   ####### base 2006-2019 pour la suite des analyses
#...........................................................................................
#................................................................................
## Choix de la p?riode d'?tude et de l'ann?e d'edi  avec la base Geo

Geo <- Geo %>%
  mutate(IRIS = sprintf("%09.0f",as.numeric(IRIS)))

library(dplyr)
library(ggplot2)
Geo$IRIS <- as.numeric(Geo$IRIS)

##Les cas non g?ocod? dans les

sans_iris <- Geo %>%
  filter(is.na(IRIS)|IRIS == "")

library(openxlsx)
write.xlsx(sans_iris,"cas non geocodes periode .xlsx")

## on va extraire pour les d?partements

sans_iris <- sans_iris %>%
  mutate(departement = case_when(
    grepl("^68P",IDPAT_KEA) ~ "68",
    grepl("^67P",IDPAT_KEA) ~ "67",
    TRUE ~ "Autre"
  ))
table_sans_iris <- sans_iris %>%
  group_by(T_INCIDENCE_DATE, departement) %>%
  summarise(n = n(), .groups = "drop") %>%
  group_by(T_INCIDENCE_DATE) %>%
  mutate( prop = round( 100 * n / sum(n),1)) %>%
  ungroup()
## Histogramme

ggplot(table_sans_iris,aes(x = T_INCIDENCE_DATE, y = n,fill = departement)) +
  geom_col(position = "dodge") +
  geom_text(aes(label = paste0(prop, "%")),
            position = position_dodge(width =  0.9),
            vjust = -0.5, size = 3.5) +
  labs(
    title = "Nombre et proportion de cas sans IRIS par d?partement et par ann?e 2006-2019",
    x = "Ann?e d'incidence",
    y = "Nombre de cas",
    fill ="D?partement"
  ) +
  theme_minimal()

##Avec tableau simplifi?

library(dplyr)
GG <- Geo %>%
  mutate(geo_ok = !is.na(IRIS) & IRIS != "") %>%
  mutate(departement = substr(IDTUM_KEA, 1,2)) %>%
  group_by(departement,T_INCIDENCE_DATE) %>%
  summarise(
    total_cas = n(),
    cas_non_geocode = sum(!geo_ok),
    .groups = "drop"
  ) %>%
  mutate(
    pct_total = 100,
    pct_non_geocode = round(100* cas_non_geocode/total_cas,2)
  ) %>%
  arrange(departement,T_INCIDENCE_DATE)

library(openxlsx)
write.xlsx(GG,"cas non geocode.xlsx")
## Recoder les A_EDI de 2006-2009 AVEC la version edi 2007

Geo <- Geo %>%
  mutate(
    A_EDI = case_when(
      !is.na(IRIS) & IRIS !="" & T_INCIDENCE_DATE >= 2006 & T_INCIDENCE_DATE <= 2009 ~ 2007L,
      TRUE ~ as.integer(A_EDI)
    )
  )

library(dplyr)
library(stringr)

Geo <- Geo %>%
  mutate(dep = case_when(
    str_detect(IDPAT_KEA, "^67P") ~ "67",
    str_detect(IDPAT_KEA, "^68P") ~ "68",
    TRUE ~ "Autre"
  ))

Geo$dep <- as.numeric(Geo$dep)


total_dep <-Geo %>%
  filter(dep %in% c("67","68")) %>%
  group_by(dep) %>%
  summarise(total_dep = n())

##Pourcentage d'utilisation des Ann?e d'EDI par d?partement

edi_summary_dep <- Geo %>%
  mutate(edi_cat = case_when(
    A_EDI == 2007 ~ "EDI 2007",
    A_EDI == 2011 ~ "EDI 2011",
    is.na(A_EDI) ~ "Manquant"
  )) %>%
  filter(dep %in% c("67", "68")) %>%
  group_by(dep, edi_cat) %>%
  summarise(n = n(),.groups = "drop") %>%
  left_join(total_dep,by = "dep") %>%
  mutate(pourcentage = round((n/ total_dep)*100,2))

write.xlsx(edi_summary_dep,"Version_edi.xlsx")
library(dplyr)

# R?sum? des p?riodes par d?partement et version deA_EDI

periode_edi <- Geo %>%
  filter(!is.na(A_EDI)) %>%
  group_by(dep,A_EDI) %>%
  summarise(
    min_inci = min(T_INCIDENCE_DATE, na.rm = TRUE),
    max_inci = max(T_INCIDENCE_DATE, na.rm = TRUE),
    n = n(),
    .groups = "drop"
  ) %>%
  arrange(dep,A_EDI)
write.xlsx(periode_edi,"periode_edi.xlsx")
#############################################################
### EDI par ann?e d'incidence

edi_par_inci_dep <- Geo %>%
  filter(!is.na(T_INCIDENCE_DATE),dep %in% c("67","68")) %>%
  mutate(edi_cat = case_when(
    A_EDI == 2007 ~ "EDI 2007",
    A_EDI == 2011 ~ "EDI 2011",
    is.na(A_EDI) ~ "Manquant"
  )) %>%
  group_by(dep,T_INCIDENCE_DATE,edi_cat) %>%
  summarise(n = n(), .groups = "drop") %>%
  group_by(dep,T_INCIDENCE_DATE) %>%
  mutate(total = sum(n),
         pourcentage = round((n / total)*100, 2)) %>%
  arrange(dep,T_INCIDENCE_DATE,edi_cat)

library(openxlsx)
write.xlsx(edi_par_inci_dep,"Edi_par_annee_Dep.xlsx") ### Pour telecharger le fichier

######### Calcul du nombre de cas par iris de 2006 ? 2019 avec la base Geo #################
############################################################################################

library(dplyr)

#Calcul du nombre de cas par iris et par ann?e

nb_cas_par_iris <- Geo %>%
  filter(T_INCIDENCE_DATE >= 2010, T_INCIDENCE_DATE <= 2017) %>%
  group_by(IRIS, T_INCIDENCE_DATE) %>%
  summarise(nb_cas = n(), .groups = "drop")


library(dplyr)

# Nettoyage du champ iris
nb_cas_par_iris %>%
  mutate(IRIS = trimws(IRIS)) %>%
  filter(IRIS == "NA" | is.na(IRIS) | IRIS == "") %>%
  group_by(T_INCIDENCE_DATE) %>%
  summarise(nb_cas_na_iris = sum(nb_cas), .groups = "drop")


library(dplyr)

# Nettoyage et calcul
Cas_na<-nb_cas_par_iris %>%
  mutate(IRIS = trimws(IRIS)) %>%
  group_by(T_INCIDENCE_DATE) %>%
  summarise(
    total_cas = sum(nb_cas),
    cas_iris_na = sum(nb_cas[IRIS == "NA" | is.na(IRIS) | IRIS == ""]),
    pourcentage = round(cas_iris_na / total_cas * 100, 2),
    .groups = "drop"
  )

library(openxlsx)
write.xlsx(Cas_na,"Nombre de cas sans iris.xlsx") ### Pour telecharger le fichier

library(dplyr)

cas_2019 <-nb_cas_par_iris %>%
  mutate(IRIS = trimws(IRIS)) %>%
  summarise(
    total_cas = sum(nb_cas),
    cas_iris_na = sum(nb_cas[IRIS == "NA" | is.na(IRIS) | IRIS == ""]),
    pourcentage = round(cas_iris_na / total_cas * 100, 2)
  )

library(openxlsx)
write.xlsx(cas_2019,"cas sans iris.xlsx") ### Pour telecharger le fichier
#Ajouter nb_cas ? chaque individu sans supprimer les colonnes

cas_par_iris <- Geo %>%
  left_join(nb_cas_par_iris, by = c("IRIS", "T_INCIDENCE_DATE"))

sapply(cas_par_iris, function(x) sum(is.na(x)))

# V?rification
View(cas_par_iris)

cas_par_iris <- cas_par_iris %>%
  mutate(IRIS = sprintf("%09.0f",as.numeric(IRIS)))

table(nchar(cas_par_iris$IRIS))


sapply(cas_par_iris, function(x) sum(is.na(x)))

##### Calcul du taux brut on va utiliser les donn?es INSEE pour l'effectif des populations

population_2006_2019 <- df_population_2006_2019 %>%
  mutate(
    IRIS = as.character(IRIS),
    Ann?e = as.character(Ann?e)
  )

cas_par_iris <- cas_par_iris %>%
  mutate(
    IRIS = as.character(IRIS),
    T_INCIDENCE_DATE = as.character(T_INCIDENCE_DATE)
  )


table(nchar(population_2006_2019$IRIS))
table(nchar(cas_par_iris$IRIS))

res_brut_ind_67_68 <-cas_par_iris %>%
  left_join(population_2006_2019, by = c("IRIS","T_INCIDENCE_DATE" = "Ann?e"))


#### Fonction pour calculer le taux brut

library(dplyr)

fusionner_et_calculer_taux <- function(population, cas) {
  cas <- cas %>%
    mutate(
      IRIS = as.character(IRIS),
      Ann?e = T_INCIDENCE_DATE
    )
  base_finale <- population %>%
    left_join(cas, by = c("IRIS", "Ann?e")) %>%
    mutate(
      nb_cas = ifelse(is.na(nb_cas), 0, nb_cas),
      taux_brut = round((nb_cas / Population_Totale) * 100000, 2)
    )

  return(base_finale)
}

#####Fonction  Base un cas par ligne pour toute la France

res_brut_ind <- fusionner_et_calculer_taux(population_2006_2019,cas_par_iris)
##############################################################################

## on va filtrer cette base sur le DEP 68

res_brut_ind_68 <- res_brut_ind %>%
  filter(DEP == "68")
res_brut_ind_67 <- res_brut_ind %>%
  filter(DEP == "67")
View(res_brut_ind_68)

res_brut_ind_dep <- res_brut_ind %>%
  filter(DEP %in% c("67","68"))         #### 1248 IRIS sans TENIR COMPTE des modifications

res_brut_ind_67_68 <- res_brut_ind_67_68 %>%
  mutate(T_brut =round((nb_cas / Population_Totale)*100000, 2))

str(population_2006_2019$IRIS)
str(res_brut_ind_67_68)

table(nchar(res_brut_ind_67_68$IRIS))

################################################################################
#################################################################################
## Evolution des iris dans le temps

library(dplyr)
library(tidyr)

population_67_68 <- population_2006_2019 %>%
  filter(DEP %in% c("67","68"))

Nombre_iris<-population_67_68 %>%
  group_by(Ann?e,DEP) %>%
  summarise(Nb = n_distinct(IRIS)) ## Nombre total d'iris pr?sent par ann?e et par d?partements


library(openxlsx)
write.xlsx(Nombre_iris,"Nombre_iris.xlsx") ### Pour telecharger le fichier


iris_annee <- population_67_68 %>%
  distinct(IRIS, Ann?e, .keep_all = TRUE)

library(tidyr)
library(dplyr)
iris_complet <- iris_annee %>%
  complete(IRIS,Ann?e) %>%
  group_by(IRIS,Ann?e) %>%
  fill(REG, DEP, COM, LIBCOM, LIBIRIS, MODIF_IRIS, Population_Totale, .direction ="downup") %>%
  ungroup()

iris_absents <- iris_complet %>%
  anti_join(population_67_68, by = c("IRIS","Ann?e"))


library(openxlsx)
write.xlsx(iris_absents,"iris_absents.xlsx") ### Pour telecharger le fichier


iris_absents %>%
  group_by(Ann?e) %>%
  summarise(Nombre_iris_absents = n_distinct(IRIS)) %>%

iris_iris  <-iris_absents %>%
  left_join(population_67_68, by = c("IRIS", "Ann?e"))

iris_absents_unique <- iris_absents %>%
  group_by(IRIS) %>%
  slice_max(order_by = Ann?e, n =1) %>%
  ungroup()

# On indique que chaque ligne correspond ? une pr?sence effective
iris_annee_complet <- iris_annee %>%
  mutate(present = 1)

### ona les IRIS absent entre  apr?s 2013 ils sont uniquement pr?sent qu'entre  2006-2013

## https://france.comersis.com/commune-nouvelle.php?dpt=68 c'est le lien internet concernant la fusion ou scission des IRIS concern?s

iris_a_suivre <- c("680310000", "680700000","680560201" ,"681080000", "681330000", "681640000","670140000","670410000","671580000","674020000","674390000","675600000","674960000",
                   "682060000", "682330000", "682720000", "683100000", "683140000", "683190000","672070000","672970000","673740000","674310000","674690000","675120000")

evolution_iris <- population_67_68 %>%
  filter(IRIS %in% iris_a_suivre) %>%
  arrange(IRIS, Ann?e)

write.csv("evolution_iris","evolution_iris.csv") #### chargement du fichier

iris_modif<-evolution_iris %>%
  group_by(IRIS) %>%
  summarise(
    debut = min(as.numeric(Ann?e)),
    fin = max(as.numeric(Ann?e)),
    statut = case_when(
      fin <= 2013 ~ "Disparu apr?s 2013",
      debut >= 2014 ~ "Apparu apr?s 2013",
      TRUE ~ "Pr?sent sur toute la p?riode"
    ),
    .groups = "drop"
  )


library(openxlsx)
write.xlsx(iris_modif,"iris_modif.xlsx") ### Pour telecharger le fichier


communes_disparues <- population_67_68 %>%
  filter(IRIS %in% iris_a_suivre) %>%
  distinct(COM)

iris_possibles_remplacants <- population_67_68 %>%
  filter(Ann?e >= 2016, COM %in% communes_disparues$COM) %>%
  distinct(IRIS, COM, LIBIRIS, Ann?e)

# V?rifier les COM absents apr?s 2013
communes_post_2013 <- population_67_68 %>%
  filter(Ann?e >= 2014) %>%
  distinct(COM, LIBCOM)

communes_disparues %>%
  anti_join(communes_post_2013, by = "COM")  ## on a le code de nos commune disparus apr?s  2014

# Extraire les couples COM x Ann?e

commune_annee <- population_67_68 %>%
  filter(DEP == "68") %>%
  distinct(COM, LIBCOM, Ann?e) %>%
  mutate(present = 1)

# Table de pr?sence par commune
table_communes <- commune_annee %>%
  pivot_wider(names_from = Ann?e, values_from = present, values_fill = 0)

#  Communes absentes avant 2014 mais pr?sentes ensuite
communes_apparues <- table_communes %>%
  filter(rowSums(select(., `2006`:`2013`)) == 0 &
           rowSums(select(., `2014`:`2019`)) > 0) %>%
  select(COM, LIBCOM)

library(openxlsx)
write.xlsx(cas_2019,"cas sans iris.xlsx") ### Pour telecharger le fichier
### L? c'est bon pour l'?volution de nos iris du 68 de 2006-2019
#################################################################
##...............................................................
## on va mettre en place une fonction qui va tenir en compte de ses changements
## cette fonction va faire le changement ? partir de 2006 et sur toutes la p?riode
## un changement ? partir de 2014 pour garder la traçabilit? de nos anciens iris

library(dplyr)

correspondance_iris <- tibble::tribble(
  ~IRIS,        ~IRIS_nouveau,
  "680310000",  "680060000",
  "680700000",  "680560201",
  "681080000",  "682400000",
  "681330000",  "682400000",
  "681640000",  "681620000",
  "682060000",  "680120000",
  "682330000",  "682010000",
  "682720000",  "683630000",
  "683100000",  "681620000",
  "683140000",  "681800000",
  "683190000",  "683200000",
  "670140000",  "674180000",
  "670410000",  "670040000",
  "671580000",  "675390000",
  "672070000",  "675390000",
  "672970000",  "675390000",
  "673740000",  "674950000",
  "674020000",  "673720000",
  "674310000",  "670040000",
  "674390000",  "672020000",
  "674690000",  "670040000",
  "674960000",  "673720000",
  "675120000",  "673720000",
  "675600000",  "671530000"
  )


### on a une base les anciens iris

res_brut_ind_68_67_hist <- res_brut_ind_67_68 %>%
  left_join(correspondance_iris, by = "IRIS") %>%
  mutate(
    IRIS_corrige = if_else(as.numeric(T_INCIDENCE_DATE) >= 2014 & !is.na(IRIS_nouveau), IRIS_nouveau, IRIS),
    statut_IRIS = case_when(
      as.numeric(T_INCIDENCE_DATE) < 2014 ~ "Ancien IRIS",
      !is.na(IRIS_nouveau) ~ "Fusionn?e (post-2014)",
      TRUE ~ "Conserv?e"
    )
  ) %>%
  select(-IRIS_nouveau)  # garde tout le reste (colonnes inchang?es)


######### les changement

res_brut_ind_68_67_fusion <- res_brut_ind_67_68 %>%
  left_join(correspondance_iris, by = "IRIS") %>%
  mutate(
    IRIS_corrige = coalesce(IRIS_nouveau, IRIS),
    statut_IRIS = if_else(!is.na(IRIS_nouveau), "Fusionn?", "Conserv?")
  ) %>%
  select(-IRIS_nouveau)



library(dplyr)
cas<-res_brut_ind_68_67_fusion %>%
  group_by(DEP) %>%
  summarise(Nb = n_distinct(IRIS_corrige))

###################################################################
#### avec cette 3?me base on va ?ffectuer les changements danss 2006
### pour assurer la coh?rence des analyses et c'est avec cette base  (res_brut_ind_68_mc) qu'on va continuer
write.csv("res_68_mc","res_68.csv")

### on va gader  uniquement 2010-2017
res_brut_ind_68_67_fusion <- res_brut_ind_68_67_fusion %>%
  filter(T_INCIDENCE_DATE >= 2010,T_INCIDENCE_DATE <= 2017)

#res_68_mc <- res_brut_ind_68_67_fusion %>%
  select(-nb_cas,-nb_cas_par_IRIS_annee)

res_brut_ind_68_mc <- res_brut_ind_67_68 %>%
  left_join(correspondance_iris, by = "IRIS") %>%
  mutate(IRIS = coalesce(IRIS_nouveau, IRIS)) %>%
  select(-IRIS_nouveau)

res_brut_ind_68_fusion <- res_brut_ind_68_67_fusion %>%
    select(-nb_cas,-taux_brut)

library(dplyr)
cas<-res_brut_ind_68_fusion %>%
  group_by(Ann?e) %>%
  summarise(IRIS_historique = n_distinct(IRIS),IRIS_fusion = n_distinct(IRIS_corrige))

library(openxlsx)
write.xlsx(cas,"Nombre_iris_annee.xlsx")

## .............................................................................
################################################################################


############################################################################
######## on va agr?ger au niveau de L'iris ###############################

library(dplyr)
library(tidyr)

# Identifier les cas valides (pr?sence des 2 identifiants) on va calculer le nombre de cas par iris
# si on a IDPAT et IDTUM pr?sent dans une ligne d'iris je consid?re comme 1 cas si c'est pas le cas je consid?re comme 0 cas
 res_brut_ind_68_67_fusion <- res_brut_ind_68_67_fusion %>%
   mutate(IRIS = format(as.character(IRIS),scientific = FALSE))

res_68_67_mc <- res_brut_ind_68_67_fusion %>%
  mutate(is_cas = if_else(!is.na(IDPAT_KEA) & !is.na(IDTUM_KEA), 1, 0))

res_brut_ind_68_67_fusion$IRIS <- sprintf("%09s", res_brut_ind_68_67_fusion$IRIS)
table(nchar(res_brut_ind_68_67_fusion$IRIS))

# 2. Agr?ger proprement
res_agrege_iris <- res_68_67_mc %>%
  group_by(IRIS_corrige, T_INCIDENCE_DATE) %>%
  summarise(
    nb_cas             = sum(is_cas, na.rm = TRUE),
    Population_Totale = first(Population_Totale),
    REG               = first(REG),
    DEP               = first(DEP),
    COM               = first(COM),
    LIBCOM            = first(LIBCOM),
    LIBIRIS           = first(LIBIRIS),
    TYP_IRIS          = first(TYP_IRIS),
    MODIF_IRIS        = first(MODIF_IRIS),
    EDI               = first(EDI),
    QUINTILE          = first(QUINTILE),
    taux_brut         = if_else(Population_Totale > 0,
                                (nb_cas / Population_Totale) * 100000,
                                NA_real_),
    .groups = "drop"
  )


View(res_agrege_iris)


library(dplyr)
cas<-res_agrege_iris %>%
  group_by(T_INCIDENCE_DATE) %>%
  summarise(Nb = n_distinct(IRIS_corrige))

res_agrege_iris <- res_agrege_iris %>%
  rename(IRIS = IRIS_corrige)
### apr?s on peut v?rifier le nombre total d'iris pr?sent pendant toute la p?riode

library(dplyr)
library(tidyr)

# 1. Extraire les Ann?es d'observation par IRIS

iris_presence <- res_68_67_mc %>%
  distinct(IRIS, T_INCIDENCE_DATE) %>%
  mutate(present = 1) %>%
  pivot_wider(names_from = T_INCIDENCE_DATE, values_from = present, values_fill = 0)

# 2. Compter le nombre d'ann?es de pr?sence

iris_presence <- iris_presence %>%
  mutate(nb_annees_presentes = rowSums(select(., -IRIS)))

# 3. Filtrer ceux pr?sents toutes les Ann?es (2006 ? 2019 = 14 ans)
iris_toujours_presents <- iris_presence %>%
  filter(nb_annees_presentes == 8)

# 4. Compter combien d'IRIS sont pr?sents sur toute la p?riode
n_toujours_presents <- nrow(iris_toujours_presents)

# ?R?sultat
print(glue::glue("Nombre d'IRIS pr?sents sur toute la p?riode (2006 ? 2019) : {n_toujours_presents}"))

#######################################################################################################
############################################################################
#######################################
######### chargement des fichier

write.csv(res_agrege_iris, "res_agrege_iris.csv", row.names = FALSE)            #### donn?e agr?g? au niveau de l'iris
# write.csv(res_68_mc, "res_68_mc", row.names = FALSE)                  #### donn?e brut au niveau individuelle
# write.csv(Geo, "Geo.csv", row.names = FALSE)          #### donn?e initiale

## l? c'est bon on passe ? l'analyse descriptive pour ça on doit coder les diff?rents localisations de Cancers

################################################################################
##########################Exploration de la Base res_68_mc ###########################

View(res_68_67_mc)

table(nchar(res_68_67_mc$IRIS))
res_68_67_mc <- res_68_67_mc %>%
  select(-IRIS)


res_68_67_mc <- res_68_67_mc %>%
  rename(IRIS = IRIS_corrige)
str(res_68_67_mc)

library(dplyr)
cas<-res_68_67_mc %>%
  group_by(T_INCIDENCE_DATE) %>%
  summarise(Nb = n_distinct(IRIS))

############## Codage selon la localisation des Cancers

####################################################################################################################################
## Fonction pour coder les diff?rentes Codes topo et histo(pas tout) sur le cancer en m?me temps filtrer les localisations ? ?tudier
#####################################################################################################################################
## les donn?es pour chaque patient chez trouve dans la base res_68_67_mc donc avec cette base on peut coder de fa?ons individuelle les codes topos

cible <- c("I_SEXE","T_TOPO","T_HISTO","QUINTILE")
names(res_68_67_mc)[names(res_68_67_mc) %in% cible]<-tolower(names(res_68_67_mc)[names(res_68_67_mc) %in% cible])

###### Fonctions pour les codes topos et morpho

library(dplyr)
library(stringr)

#
# 1. Fonction de classement topographique
#    (2 arguments : code + sexe)
#
get_topo_group <- function(code, sexe) {
  if (is.na(code)) return(NA_character_)
  code <- as.integer(code)

  if (code %in%   0:14) return("L?vre-Bouche-pharynx")
  if (code == 15) return("Oesophage")
  if (code == 16) return("Estomac")
  if (code %in%  18:21) return("Colon-Rectum")
  if (code == 22) return("Foie")
  if (code == 25) return("Pancr?as")
  if (code == 32) return("Larynx")
  if (code %in%  33:34) return("Poumon")
  if (code == 44) return("M?lanome cutan?")
  if (code == 45) return("M?soth?liome")
  if (code == 50) return("Sein")
  if (code == 53) return("Col de l'ut?rus")
  if (code == 54) return("Corps de l'ut?rus")

  # Ovaire uniquement si sexe = 2 (femme)
  if (code %in% c(48, 56, 57))
    return(ifelse(sexe == 2, "Ovaire", "Autres"))

  if (code == 61) return("Prostate")
  if (code == 62) return("Testicule")
  if (code == 64) return("Rein")
  if (code == 67) return("Vessie")
  if (code %in%  65:68) return("Autres voies urinaires")
  if (code %in%  70:72) return("Syst?me nerveux central")
  if (code == 73) return("Thyroide")

  "Autres"
}

#
# 2. Pipeline de pr?paration pour les 2 premier chiffres
#

process_df <- function(df) {
  df <- df %>%
    mutate(
      # copie ?ventuelle si vous voulez figer la valeur brute
      t_topo_brut  = t_topo,

      # code ? deux chiffres, z?ro-padding
      topo_group   = str_sub(sprintf("%03d", as.integer(t_topo)), 1, 2),

      # localisation agr?g?e (code + sexe)
      topo_grp     = mapply(get_topo_group,
                            as.integer(topo_group),
                            i_sexe)
    )

}

# 3. Utilisation de la fonction
################################################################################
########################
res_mc <- process_df(res_68_67_mc)                                    ############ ok pour l'instant
dplyr::count(res_mc, topo_grp)                              ########################
################################################################################


################################################################################
## 0 • Packages ---------------------------------------------------------------
################################################################################
library(dplyr)
library(stringr)

################################################################################
## 1 • Fonction d'origine : 4 premiers chiffres  ➜  groupe morpho  (« JAMAIS
##     supprim?e ! ») ----------------------------------------------------------
################################################################################
get_morpho_group <- function(code) {
  if (is.na(code)) return(NA_character_)
  prefix <- as.integer(substr(as.character(code), 1, 4))

  if (prefix %in% c(9650:9655, 9659, 9661:9667))   return("Lymphome de Hodgkin")
  if (prefix %in% c(9670, 9823))                   return("LLC/Lymphome lymphocytique")
  if (prefix %in% c(9690:9698, 9597))              return("Lymphome folliculaire")
  if (prefix %in% c(9678:9684, 9688, 9712, 9735,
                    9737, 9738, 9766))             return("Lymphome diffus ? grande cellules B")
  if (prefix %in% c(9731:9734))                    return("My?lome Multiple et plasmocytome")
  if (prefix %in% c(9840, 9860, 9861, 9866, 9867,
                    9870:9874, 9891:9931, 9984,
                    9805, 9806:9809, 9865, 9869,
                    9911, 9898, 9877:9879, 9912))  return("Leuc?mie aigu?? my?loide")

  if (prefix %in% c(9950, 9960:9964, 9875, 9863))  return("Syndrome My?loprolif?ratif Chronique")
  if (prefix %in% c(9980:9983, 9985:9986, 9989,
                    9991:9993))                    return("Syndrome My?lodysplasique")
  if (prefix %in% c(9876, 9945, 9946, 9975))       return("Leuc?mie my?lomonocytaire chronique et autres (SMD/SMP)")

  NA_character_
}



get_morpho_group <- function(code) {
  if (is.na(code)) return(NA_character_)
  prefix <- as.integer(substr(as.character(code), 1, 4))

  if (prefix %in% c(9650:9655, 9659, 9661:9667))   return("Lymphome de Hodgkin")
  if (prefix %in% c(9670, 9823))                   return("LLC/Lymphome lymphocytique")
  if (prefix %in% c(9690:9698, 9597))              return("Lymphome folliculaire")
  if (prefix %in% c(9678:9684, 9688, 9712, 9735, 9737, 9738, 9766)) return("Lymphome diffus ? grande cellules B")
  if (prefix %in% c(9731:9734))                    return("My?lome Multiple et plasmocytome")
  if (prefix %in% c(9840, 9860, 9861, 9866, 9867,
                    9870:9874, 9891:9931, 9984,
                    9805, 9806:9809, 9865, 9869,
                    9911, 9898, 9877:9879, 9912))  return("Leuc?mie aigu?? my?loide")
  if (prefix %in% c(9950, 9960:9964, 9875, 9863))  return("Syndrome My?loprolif?ratif Chronique")
  if (prefix %in% c(9980:9983, 9985:9986, 9989, 9991:9993)) return("Syndrome My?lodysplasique")
  if (prefix %in% c(9876, 9945, 9946, 9975))       return("Leuc?mie my?lomonocytaire chronique et autres (SMD/SMP)")
  if (prefix %in% c(8720:8780))                    return("M?lanome cutan?")

  return(NA_character_)
}




################################################################################
## 2 • Option : garder, quand n?cessaire, le code OMS ? 5 chiffres ------------
################################################################################

get_morpho_group_keep_5digits <- function(code) {
  if (is.na(code)) return(NA_character_)
  prefix <- as.integer(substr(as.character(code), 1, 4))

  valid_prefixes <- c(
    9590:9591, 9597, 9670:9699, 9700:9719, 9724:9729,
    9731:9734, 9735, 9737, 9738, 9760:9764, 9766, 9768,
    9688, 9712, 9811:9818, 9820, 9823, 9826, 9827, 9831:9837, 9940, 9948,8720:8780
  )

  if (prefix %in% valid_prefixes) as.character(code) else NA_character_
}


################################################################################
## 3 • Pipeline complet  -------------------------------------------------------
################################################################################
################################################################################
# 1. Ajout des variables topographiques
################################################################################

add_topo <- function(df) {
  df %>%
    mutate(
      t_topo_brut = t_topo,
      topo_group  = str_sub(sprintf("%03d", as.integer(t_topo)), 1, 2),
      topo_grp    = mapply(get_topo_group,
                           as.integer(topo_group),
                           i_sexe)
    )
}


add_topo <- function(df) {
  df %>%
    mutate(
      t_topo_brut = t_topo,
      topo_group  = str_sub(sprintf("%03d", as.integer(t_topo)), 1, 2),
      topo_grp    = mapply(get_topo_group, as.integer(topo_group), i_sexe)
    )
}
################################################################################
# 2. Ajout / priorisation morphologique
################################################################################

add_morpho <- function(df, codes_histo) {
  lnh_sub <- c("LLC/Lymphome lymphocytique",
               "Lymphome folliculaire",
               "Lymphome diffus ? grande cellules B")

  #
  lookup <- codes_histo %>%
    filter(as.integer(substr(Valeur, 1, 4)) %in% 8720:8790) %>%
    select(Valeur, Libell?)

  df %>%
    mutate(
      morpho_grp4 = sapply(t_histo, get_morpho_group),
      morpho_5    = sapply(t_histo, get_morpho_group_keep_5digits),
      morpho_grp  = if_else(is.na(morpho_grp4) & !is.na(morpho_5),
                            morpho_5, morpho_grp4),

      morpho_grp = case_when(
        morpho_grp %in% lnh_sub             ~ "Lymphomes non hodgkiniens",
        str_detect(morpho_grp, "^\\d{5}$")  ~ "Lymphomes non hodgkiniens",
        morpho_grp == "My?lome Multiple et plasmocytome" ~ "My?lome",
        TRUE ~ morpho_grp
      ),

      code_topo   = paste0("C", str_pad(t_topo, 2, pad = "0")),
      code_morpho = as.integer(substr(t_histo, 1, 4)),

      #  Correction sp?cifique : ne garder que les m?lanomes cutan?s
      topo_final = case_when(
        code_topo == "C44" & code_morpho %in% 8720:8790 ~ "M?lanome cutan?",
        code_topo == "C44" & !code_morpho %in% 8720:8790 ~ NA_character_,
        TRUE ~ NA_character_
      )
    ) %>%

    mutate(
      topo_final = case_when(
        is.na(topo_final) & morpho_grp == "Lymphomes non hodgkiniens" ~ "Lymphomes non hodgkiniens",
        is.na(topo_final) & morpho_grp == "My?lome"                   ~ "My?lome",
        is.na(topo_final)                                             ~ topo_grp,
        TRUE ~ topo_final
      ),

      morpho_final = case_when(
        code_topo == "C44" & code_morpho %in% 8720:8790 ~ "M?lanome cutan?",
        str_detect(morpho_grp, "^\\d{5}$") ~ lookup$Libell?[match(morpho_grp, lookup$Valeur)],
        TRUE ~ morpho_grp
      )
    )
}

################################################################################
# 3. Cha?nage
################################################################################
df_localisation <- res_mc %>%
  add_topo() %>%             # cr?e topo_grp
  add_morpho(CODES_HISTO)    # cr?e topo_final & morpho_final

# Comptes
count(df_localisation, topo_final)
count(df_localisation, morpho_final, topo_final)


library(dplyr)
cas<-df_localisation %>%
  group_by(DEP) %>%
  summarise(Nb = n_distinct(IRIS))

##################################################

## Probl?me sur le m?lanome r?solu avec ce code

df_localisation <- df_localisation %>%
  mutate(topo_final = if_else(code_morpho >= 8720 & code_morpho <= 8780,
                              "M?lanome",
                              topo_final))


df_localisation <- df_localisation %>%
  filter(topo_final != "M?lanome cutan?")

df_localisation <- df_localisation %>%
  mutate(topo_final = if_else(topo_final == "M?lanome", "M?lanome cutan?", topo_final))


#################################################################################
###### Histogramme #############################################################
## Distribution de l'?ge au diagnostic

library(tidyverse)

str(df_localisation)
summary(df_localisation)

df_localisation <- df_localisation %>%
  rename(Ann?e = T_INCIDENCE_DATE)

## on va calculer le nombre de cas pour chaque type de cancer par iris et par Ann?e

library(dplyr)
library(tidyverse)
# V?rifie que "is_cas" existe (qui nous donne si un iris contient un cas ou pas)

df_localisation <- df_localisation %>%
  mutate(
    is_cas = if_else(!is.na(IDPAT_KEA) & !is.na(IDTUM_KEA), 1, 0)
  )


# tous les types de cancer possibles

types_cancer <- unique(na.omit(df_localisation$topo_final))

#toutes les combinaisons IRIS-Ann?e-Type (m?me absents)
grille <- df_localisation %>%
  group_by(IRIS, Ann?e) %>%
  slice(1) %>%
  ungroup() %>%
  select(IRIS, Ann?e, REG, DEP, COM, LIBCOM, LIBIRIS, TYP_IRIS, Population_Totale) %>%
  distinct() %>%
  crossing(topo_final = types_cancer)


cas_cancer <- df_localisation %>%
  filter(!is.na(topo_final)) %>%
  group_by(IRIS, Ann?e, topo_final) %>%
  summarise(n_cas = sum(is_cas, na.rm = TRUE), .groups = "drop")

table_iris_cancer <- grille %>%
  left_join(cas_cancer, by = c("IRIS", "Ann?e", "topo_final")) %>%
  mutate(n_cas = replace_na(n_cas, 0))


library(tidyverse)
library(moments)  # pour skewness et kurtosis

# Calcul de l'?ge au diagnostic
df_localisation <- df_localisation %>%
  mutate(I_AGE = floor(as.numeric(I_AGE)))

# Supprimer les valeurs manquantes
ages <- na.omit(df_localisation$I_AGE)

# Statistiques descriptives
moyenne <- mean(ages)
ecart_type <- sd(ages)
skew <- skewness(ages)
kurt <- kurtosis(ages) - 3  # kurtosis excessive
min_val <- min(ages)
max_val <- max(ages)

# Donn?es pour la courbe normale th?orique
densite_theorique <- tibble(
  x = seq(min_val, max_val, length.out = 200),
  y = dnorm(seq(min_val, max_val, length.out = 200), mean = moyenne, sd = ecart_type)
)

# ?chelle pour superposer correctement la densit? sur l'histogramme (en nombre de cas)
scaling_factor <- length(ages) * 5  # 5 = largeur des bins (binwidth)
densite_theorique <- densite_theorique %>%
  mutate(y = y * scaling_factor)

# Graphique final

ggplot(data = data.frame(ages), aes(x = ages)) +
  geom_histogram(binwidth = 5, fill = "orange", color = "white") +
  geom_density(aes(y = ..density.. * scaling_factor), color = "blue", size = 1) +
  geom_line(data = densite_theorique, aes(x = x, y = y),
            color = "grey40", linetype = "dashed", size = 1) +
  scale_x_continuous(breaks = seq(0, 105, by = 10)) +
  labs(
    title = "Distribution de l'?ge au diagnostic - Tous cancers (Alsace, 2010-2017)",
    x = "?ge",
    y = "Nombre de cas"
  ) +
  annotate("label",
           x = min_val + 5,
           y = max(table(cut(ages, breaks = seq(min_val, max_val, by = 5)))) * 0.9,
           hjust = 0,
           label = paste0("Moyenne      = ", round(moyenne, 2), "\n",
                          "?cart-type   = ", round(ecart_type, 2), "\n",
                          "Maximum      = ", max_val, "\n",
                          "Minimum      = ", min_val, "\n",
                          "Skewness     = ", round(skew, 5), "\n",
                          "Kurtosis     = ", round(kurt, 2)),
           size = 4.5,
           fontface = "bold",
           color = "black",
           fill = "white",      # Fond blanc
           label.size = 0.5,    # Bordure du cadre
           label.r = unit(0.15, "lines")) +  # Arrondi des coins
  theme_minimal(base_size = 14) +
  theme(
    plot.title = element_text(face = "bold", hjust = 0.5, size = 16),
    axis.title = element_text(size = 14),
    axis.text = element_text(size = 12)
  )
##...............................................................................
### (les localisations  des cancers les plus fr?quents dans notre base)

library(tidyverse)

# Calcul du total des cas
#total_cases <- sum(df_localisation$topo_final %>% table())

# Pr?paration des donn?es avec proportions avec les NA

df_plot <- df_localisation %>%
  filter(!is.na(topo_final)) %>%
  count(topo_final, sort = TRUE) %>%
  #top_n(23, n) %>%
  mutate(pct = round(n / sum(n) * 100, 1))  # % dans les 22 les plus fr?quents

total_cases <- df_localisation %>%
  filter(!is.na(topo_final)) %>%
  count(topo_final) %>%
  summarise(total = sum(n)) %>%
  pull(total)
# Affichage avec ggplot

library(ggplot2)

ggplot(df_plot, aes(x = reorder(topo_final, n), y = n)) +
  geom_col(fill = "steelblue") +
  geom_text(
    aes(label = paste0(n, " (", pct, "%)")),
    hjust = -0.1,
    size = 3
  ) +
  coord_flip(clip = "off") +
  labs(
    title = "localisations de cancer les plus fr?quents en Alsace  2010-2017",
    x = "Type de cancer",
    y = "Nombre de cas"
  ) +
  theme_minimal(base_size = 12) +
  theme(
    plot.title = element_text(face = "bold", size = 14, hjust = 0.5),
    plot.margin = margin(10, 40, 10, 10),  #
    axis.text.y = element_text(size = 9)
  ) +
  scale_y_continuous(expand = expansion(mult = c(0, 0.15)))  #



##..............................................................................
##### R?partition par sexe pour chaque localisation
#################################################################################
## le bon format ################################################################

library(tidyverse)
types_cancers_selectionnes <- c("Poumon","Sein","Prostate","M?lanome cutan?") ### Pour choisir ce que je veux afficher

df_sexe_miroir <- df_localisation %>%
  filter(!is.na(topo_final), !is.na(i_sexe)) %>%
  mutate(i_sexe = factor(i_sexe, levels = c(1, 2), labels = c("Hommes", "Femmes"))) %>%
  count(topo_final, i_sexe) %>%
  group_by(topo_final) %>%
  mutate(total = sum(n)) %>%
  ungroup() %>%
  top_n(45, total) %>%
  group_by(topo_final) %>%
  mutate(pourcent = round(n / sum(n) * 100, 1)) %>%
  ungroup() %>%
  mutate(n_miroir = ifelse(i_sexe == "Hommes", -n, n))


#### Cancers par sexe

library(openxlsx)
write.xlsx(df_sexe_miroir,"cancers_par_sexe.xlsx") ### Pour telecharger le fichier


df_sexe_miroir <- df_sexe_miroir %>%
  filter(topo_final %in% types_cancers_selectionnes) ##### ici on peut appliquer le filtre

ggplot(df_sexe_miroir, aes(x = reorder(topo_final, total), y = n_miroir, fill = i_sexe)) +
  geom_col(width = 0.7) +
  geom_text(
    aes(label = paste0(abs(n), "\n(", pourcent, "%)")),
    position = position_dodge(width = 0.9),
    hjust = ifelse(df_sexe_miroir$i_sexe == "Hommes", 1.05, -0.05),
    size = 3.2
  ) +
  coord_flip(clip = "off") +
  scale_y_continuous(
    labels = abs,
    expand = expansion(mult = c(0.1, 0.2))
  ) +
  scale_fill_manual(values = c("Hommes" = "#d7191c", "Femmes" = "#2c7bb6")) +
  labs(
    title = "Les localisations de cancer par sexe Alsace (2010-2017)",
    x = "Types de cancer",
    y = "Nombre de cas",
    fill = "Sexe"
  ) +
  theme_minimal(base_size = 12) +
  theme(
    plot.title = element_text(face = "bold", size = 15, hjust = 0.5),
    axis.text.y = element_text(size = 9.5),
    panel.grid.major.y = element_blank(),
    plot.margin = margin(t = 10, r = 60, b = 10, l = 10)
  )


#################################################################################
##..............................................................................

# pour conna?tre le nombre d'IRIS diff?rents pr?sents par Ann?e :

library(dplyr)

table_iris_cancer %>%
  distinct(IRIS, Ann?e) %>%
  count(Ann?e, name = "nb_IRIS")
write.csv(table_iris_cancer,"table_iris_cancer.csv")

#### Donc on a 1224 Iris pr?sente pour chaque Ann?e sur le fond de carte du 68-67
##############################################################################
##############################################################################
## Proportion D'IRIS avec 0 cas par ann?e tous cancers

library(dplyr)

res_agrege_iris <-res_agrege_iris %>%
  rename(Ann?e = T_INCIDENCE_DATE)

prop_par_annee <- res_agrege_iris %>%
  mutate(type_IRIS = if_else(nb_cas ==0, "0 cas", "? 1 cas")) %>%
  group_by(Ann?e, type_IRIS) %>%
  summarise(n = n(), .groups = "drop") %>%
  group_by(Ann?e) %>%
  mutate(
    proportion = round(n / sum(n) * 100, 1)
  )

write.csv(prop_par_annee,"proportion de cas par ann?e.csv")
print(prop_par_annee)

library(ggplot2)

ggplot(prop_par_annee, aes(x = factor(Ann?e), y = proportion, fill = type_IRIS)) +
  geom_col(position = "stack") +
  geom_text(aes(label = paste0(proportion, "%")),
            position = position_stack(vjust = 0.5), color = "white", size = 3.5) +
  scale_fill_manual(values = c("0 cas" = "#d73027", "? 1 cas" = "#1a9850")) +
  labs(
    title = "Proportion d'IRIS ? 0 cas et ? 1 cas par ann?e tous cancers (Alsace)2010-2017",
    x = "Ann?e",
    y = "Proportion (%)",
    fill = "Statut des IRIS"
  ) +
  theme_minimal()

#################################################################################
#################################################################################
##  Regrouper tous les cas par IRIS sur toute la p?riode 2006-2019

prop_totale <- res_agrege_iris %>%
  group_by(IRIS) %>%
  summarise(nb_total_cas = sum(nb_cas), .groups = "drop") %>%
  mutate(type_IRIS = if_else(nb_total_cas == 0, "0 cas", " >=1 cas")) %>%
  count(type_IRIS) %>%
  mutate(
    proportion = round(n / sum(n) * 100, 1)
  )

print(prop_totale)

library(openxlsx)
write.xlsx(prop_totale,"Proportion toal de cas.xlsx")

ggplot(prop_totale, aes(x = "", y = proportion, fill = type_IRIS)) +
  geom_col(width = 0.6) +
  geom_text(aes(label = paste0(proportion, "%")),
            position = position_stack(vjust = 0.5), color = "white", size = 4.5) +
  scale_fill_manual(values = c("0 cas" = "#d73027", " >=1 cas" = "#1a9850")) +
  labs(
    title = "Proportion des IRIS ? 0 cas et des IRIS ? 1  cas tous cancers (2010 ? 2017)",
    x = NULL, y = "Proportion (%)", fill = "Statut des IRIS"
  ) +
  theme_minimal() +
  theme(axis.text.x = element_blank(), axis.ticks.x = element_blank())

################################################################################
################################################################################
### Proportions par types de cancers 2006-2019

library(dplyr)
library(tidyr)

taux_absence_cancer_global <- table_iris_cancer %>%
  group_by(IRIS, Type_Cancer = topo_final) %>%
  summarise(nb_cas_total = sum(n_cas, na.rm = TRUE), .groups = "drop") %>%
  group_by(Type_Cancer) %>%
  summarise(
    nb_IRIS_absents = sum(nb_cas_total == 0),
    nb_IRIS_total = n(),
    proportion_absence = round(nb_IRIS_absents / nb_IRIS_total * 100, 2),
    .groups = "drop"
  )

print(taux_absence_cancer_global)


library(ggplot2)

# Tri croissant par proportion
donnees_barres_global <- taux_absence_cancer_global %>%
  arrange(proportion_absence) %>%
  mutate(
    Type_Cancer = factor(Type_Cancer, levels = unique(Type_Cancer)),
    label = paste0(proportion_absence, "% (", nb_IRIS_absents, ")")
  )

# le nombre total d'IRIS
nb_total_IRIS_global <- unique(donnees_barres_global$nb_IRIS_total)

# Graphique
ggplot(donnees_barres_global, aes(x = proportion_absence, y = Type_Cancer)) +
  geom_col(fill = "#2c7fb8") +
  geom_text(
    aes(label = label),
    hjust = -0.1, size = 4, fontface = "bold"
  ) +
  labs(
    title = "Proportion d'IRIS avec 0 cas observ? par type de cancer (Alsace2010-2017)",
    subtitle = paste("Nombre total d'IRIS :", nb_total_IRIS_global),
    x = "Proportion d'IRIS sans cas (%)",
    y = "Type de cancer"
  ) +
  coord_cartesian(xlim = c(0, 100)) +
  theme_minimal()

##############################################################
### Proportions par types de cancers pour chaque Ann?e

taux_absence_cancer <- table_iris_cancer %>%
  group_by(Ann?e, topo_final) %>%
  summarise(
    nb_IRIS_total = n_distinct(IRIS),
    nb_IRIS_absents = n_distinct(IRIS[n_cas == 0]),
    proportion_absence = round(100 * nb_IRIS_absents / nb_IRIS_total, 1),
    .groups = "drop"
  )

# Ann?e cible
annee_selection <- 2012  ###### si on veux afficher pour une Ann?e il suffit d'?crire l'Ann?e

# Filtrer l'Ann?e s?lectionn?e
donnees_barres <- taux_absence_cancer %>%
  filter(Ann?e == annee_selection) %>%
  arrange(proportion_absence) %>%
  mutate(
    Type_Cancer = factor(topo_final, levels = unique(topo_final)),
    label = paste0(proportion_absence, "% (", nb_IRIS_absents, ")")
  )

# Nombre total d'IRIS dans l'ann?e
nb_total_IRIS_annee <- unique(donnees_barres$nb_IRIS_total)
if (length(nb_total_IRIS_annee) > 1) nb_total_IRIS_annee <- nb_total_IRIS_annee[1]

# Graphique
ggplot(donnees_barres, aes(x = proportion_absence, y = Type_Cancer)) +
  geom_col(fill = "#2c7fb8") +
  geom_text(
    aes(label = label),
    hjust = -0.1,
    size = 4,
    fontface = "bold"
  ) +
  coord_cartesian(xlim = c(0, 100)) +
  labs(
    title = paste("Proportion d'IRIS sans cas par type de cancer ?", annee_selection),
    subtitle = paste("Nombre total d'IRIS :", nb_total_IRIS_annee),
    x = "Proportion d'IRIS sans cas (%)",
    y = "Type de cancer"
  ) +
  theme_minimal()

##..la c'est bon pour l'analyse Decriptive .et aussi la proportion des iris avec 0 cas ..
#########################################################################################
#########################################################################################
#########################################################################################
#########################################################################################
#########################################################################################



##################################### Cartographie 68-67 ###################################

### Avant de calculer nos SIR on va  Harmoniser les noms de colonnes de population IRIS pour ajouter les tranches d'?ges
### et d?coup? nos cas incidence en tranche d'?ges pour bien faciliter l'analyse

library(tidyr)

library(dplyr)

df_pop_long <- df_population_tranches %>%
  select(IRIS, REG, DEP, COM, LIBCOM, LIBIRIS, TYP_IRIS, MODIF_IRIS, Annee,
         POP0014, POP1529, POP3044, POP4559, POP6074, POP75P) %>%
  pivot_longer(
    cols = starts_with("POP"),
    names_to = "class_age_code",
    values_to = "pop_iris"
  ) %>%
  mutate(
    class_age = recode(class_age_code,
                       "POP0014" = "[0-14]",
                       "POP1529" = "[15-29]",
                       "POP3044" = "[30-44]",
                       "POP4559" = "[45-59]",
                       "POP6074" = "[60-74]",
                       "POP75P"  = ">=75")
  ) %>%
  select(-class_age_code)

## on va mettre ? jour notre df_pop_long pour les changements d'iris

library(dplyr)

# Table de correspondance IRIS fusionn?s ? IRIS valides

correspondance_iris <- tibble::tribble(
  ~IRIS,        ~IRIS_nouveau,
  "680310000",  "680060000",
  "680700000",  "680560201",
  "681080000",  "682400000",
  "681330000",  "682400000",
  "681640000",  "681620000",
  "682060000",  "680120000",
  "682330000",  "682010000",
  "682720000",  "683630000",
  "683100000",  "681620000",
  "683140000",  "681800000",
  "683190000",  "683200000",
  "670140000",  "674180000",
  "670410000",  "670040000",
  "671580000",  "675390000",
  "672070000",  "675390000",
  "672970000",  "675390000",
  "673740000",  "674950000",
  "674020000",  "673720000",
  "674310000",  "670040000",
  "674390000",  "672020000",
  "674690000",  "670040000",
  "674960000",  "673720000",
  "675120000",  "673720000",
  "675600000",  "671530000"
)


# Application ? La base populationnelle Pour tenir en compte des changements

df_pop_long_corrige <- df_pop_long %>%
  left_join(correspondance_iris, by = "IRIS") %>%
  mutate(
    IRIS_corrige = coalesce(IRIS_nouveau, IRIS),
    statut_IRIS  = if_else(!is.na(IRIS_nouveau), "Fusionn?", "Conserv?")
  ) %>%
  select(-IRIS_nouveau)



library(dplyr)

df_pop_agrege_complet <- df_pop_long_corrige %>%
  group_by(IRIS_corrige, Annee, class_age) %>%
  summarise(
    pop_iris = sum(pop_iris, na.rm = TRUE),
    .groups = "drop"
  ) %>%
  left_join(
    df_pop_long_corrige %>%
      select(IRIS, IRIS_corrige, REG, DEP, COM, LIBCOM, LIBIRIS, TYP_IRIS, MODIF_IRIS) %>%
      distinct(IRIS_corrige, .keep_all = TRUE),
    by = "IRIS_corrige"
  ) %>%
  relocate(REG, DEP, COM, LIBCOM, LIBIRIS, TYP_IRIS, MODIF_IRIS, .after = IRIS_corrige)


## c'est avec df_pop_agrge_complet et IRIS_corrige que je vais travailler pour la suite des analyses

## on va telecharger la base insee avec les tranches d'?ges
###############################################################################
library(openxlsx)
write.xlsx(df_pop_agrege_complet ,"df_pop_agrege_complet_insee.xlsx")
##############################################################################
## Cas d'incidence agrge par tranches d'?ges
## ON vas calculer le nombre de cas observ? par tranches d'?ges dans chaque iris
###############################################################################
##############################################################################
######### ici par sexe et par tranche d'?ges
#### ON VA extraire d'abord les populations par tranches d'?ges et en suite par sexe

library(dplyr)
library(tidyr)

# 1. Tableau long par sexe
df_pop_sexe <- df_population_tranches %>%
  dplyr::select(IRIS, REG, DEP, COM, LIBCOM, LIBIRIS, TYP_IRIS, MODIF_IRIS, Annee,
                matches("^[HF][0-9]{4}$|^[HF]75P$")) %>%
  pivot_longer(
    cols = matches("^[HF][0-9]{4}$|^[HF]75P$"),
    names_to = "var_code",
    values_to = "pop_sexe"
  ) %>%
  mutate(
    sexe = if_else(substr(var_code, 1, 1) == "H", "Homme", "Femme"),
    class_age = recode(substr(var_code, 2, 5),
                       "0014" = "[0-14]",
                       "1529" = "[15-29]",
                       "3044" = "[30-44]",
                       "4559" = "[45-59]",
                       "6074" = "[60-74]",
                       "75P"  = ">=75")
  ) %>%
  dplyr::select(-var_code)

# 2. Tableau total (population tous sexes, par classe dage)
df_pop_total <- df_population_tranches %>%
  dplyr::select(IRIS, Annee, POP0014, POP1529, POP3044, POP4559, POP6074, POP75P) %>%
  pivot_longer(
    cols = starts_with("POP"),
    names_to = "var_code",
    values_to = "pop_iris"
  ) %>%
  mutate(
    class_age = recode(var_code,
                       "POP0014" = "[0-14]",
                       "POP1529" = "[15-29]",
                       "POP3044" = "[30-44]",
                       "POP4559" = "[45-59]",
                       "POP6074" = "[60-74]",
                       "POP75P"  = ">=75")
  ) %>%
  dplyr::select(-var_code)

# 3. Fusion propre
df_pop_long_sexe <- df_pop_sexe %>%
  left_join(df_pop_total, by = c("IRIS", "Annee", "class_age"))


## on va mettre ? jour notre df_pop_long pour les Iris supprim?

library(dplyr)

# Table de correspondance IRIS fusionn?s  IRIS valides


correspondance_iris <- tibble::tribble(
  ~IRIS,        ~IRIS_nouveau,
  "680310000",  "680060000",
  "680700000",  "680560201",
  "681080000",  "682400000",
  "681330000",  "682400000",
  "681640000",  "681620000",
  "682060000",  "680120000",
  "682330000",  "682010000",
  "682720000",  "683630000",
  "683100000",  "681620000",
  "683140000",  "681800000",
  "683190000",  "683200000",
  "670140000",  "674180000",
  "670410000",  "670040000",
  "671580000",  "675390000",
  "672070000",  "675390000",
  "672970000",  "675390000",
  "673740000",  "674950000",
  "674020000",  "673720000",
  "674310000",  "670040000",
  "674390000",  "672020000",
  "674690000",  "670040000",
  "674960000",  "673720000",
  "675120000",  "673720000",
  "675600000",  "671530000"
)

# Application ? La base populationnelle Pour tenir en compte des changements

df_pop_long_corrige_sexe <- df_pop_long_sexe %>%
  left_join(correspondance_iris, by = "IRIS") %>%
  mutate(
    IRIS_corrige = coalesce(IRIS_nouveau, IRIS),
    statut_IRIS  = if_else(!is.na(IRIS_nouveau), "Fusionn?", "Conserv?")
  ) %>%
  dplyr::select(-IRIS_nouveau)


library(dplyr)  #  charger toujours en dernier

## on va telecharger la base insee avec les tranches d'?ges
###############################################################################
library(openxlsx)
write.xlsx(df_pop_long_corrige_sexe ,"df_pop_long_corrige_sexe.xlsx")
##############################################################################


####### SIR brut:lisse tous cancers 2006-2019 ###############################

# decoupage de nos cas par tranche d'ages

ages_vals <- c("[0-14]", "[15-29]", "[30-44]", "[45-59]", "[60-74]", ">=75")
iris_ref <- df_localisation %>%
  distinct(IRIS, REG, DEP, COM, LIBCOM, LIBIRIS, TYP_IRIS, MODIF_IRIS, quintile, Ann?e, Population_Totale)

grille_complete <- expand.grid(
  IRIS = unique(iris_ref$IRIS),
  Ann?e = unique(iris_ref$Ann?e),
  class_age = ages_vals,
  stringsAsFactors = FALSE
) %>%
  left_join(iris_ref, by = c("IRIS", "Ann?e"))

df_localisation$Ann?e <- as.numeric(df_localisation$Ann?e)
df_localisation$I_A_NAISS <- as.integer(df_localisation$I_A_NAISS)

df_localisation_age <- df_localisation %>%
  mutate(
    age_diag = Ann?e-I_A_NAISS,
    class_age = case_when(
      between(age_diag, 0, 14)   ~ "[0-14]",
      between(age_diag, 15, 29)  ~ "[15-29]",
      between(age_diag, 30, 44)  ~ "[30-44]",
      between(age_diag, 45, 59)  ~ "[45-59]",
      between(age_diag, 60, 74)  ~ "[60-74]",
      age_diag >= 75             ~ ">=75",
      TRUE ~ NA_character_
    )
  ) %>%
  filter(!is.na(class_age))

df_cas_aggr <- df_localisation_age %>%
  group_by(IRIS, Ann?e, class_age) %>%
  summarise(nb_cas = n(), .groups = "drop")


grille_complete$Ann?e <- as.numeric(grille_complete$Ann?e)
df_cas_observ? <- grille_complete %>%
  left_join(df_cas_aggr, by = c("IRIS", "Ann?e", "class_age")) %>%
  mutate(nb_cas = replace_na(nb_cas, 0)) %>%
  arrange(IRIS, Ann?e, class_age)

n_distinct(df_localisation_age$IRIS)
## dans df_cas_observ? on a les cas par tranches d'?ges ,par iris et par ann?e
## mais moi je veux afficher pour toutes la p?riode en m?me temps dans chaque iris

############################################################################
###############################################################################

ages_vals <- c("[0-14]", "[15-29]", "[30-44]", "[45-59]", "[60-74]", ">=75")

iris_ref <- df_localisation %>%
  distinct(IRIS, REG, DEP, COM, LIBCOM, LIBIRIS, TYP_IRIS, MODIF_IRIS, Population_Totale, quintile)

grille_periode <- expand.grid(
  IRIS = unique(iris_ref$IRIS),
  class_age = ages_vals,
  stringsAsFactors = FALSE
) %>%
  left_join(iris_ref, by = "IRIS")


df_total_cas <- df_localisation %>%
  mutate(
    age_diag = Ann?e - I_A_NAISS,
    class_age = case_when(
      between(age_diag, 0, 14)   ~ "[0-14]",
      between(age_diag, 15, 29)  ~ "[15-29]",
      between(age_diag, 30, 44)  ~ "[30-44]",
      between(age_diag, 45, 59)  ~ "[45-59]",
      between(age_diag, 60, 74)  ~ "[60-74]",
      age_diag >= 75             ~ ">=75",
      TRUE ~ NA_character_
    )
  ) %>%
  filter(!is.na(class_age)) %>%
  group_by(IRIS, class_age) %>%
  summarise(nb_cas_total = n(), .groups = "drop")

##### on va nettoyer avant de joindre
library(stringr)
grille_periode <- grille_periode %>%
  mutate(class_age = case_when(
    class_age == "[0-14]" ~ "[0-14]",
    class_age == "[15-29]" ~ "[15-29]",
    class_age == "[30-44]" ~ "[30-44]",
    class_age == "[45-59]" ~ "[45-59]",
    class_age == "[60-74]" ~ "[60-74]",
    str_detect(class_age,">=75") ~ ">=75",
    TRUE ~ class_age

  ))

library(stringr)
df_total_cas <- df_total_cas %>%
  mutate(class_age = case_when(
    class_age == "[0-14]" ~ "[0-14]",
    class_age == "[15-29]" ~ "[15-29]",
    class_age == "[30-44]" ~ "[30-44]",
    class_age == "[45-59]" ~ "[45-59]",
    class_age == "[60-74]" ~ "[60-74]",
    str_detect(class_age,">=75") ~ ">=75",
    TRUE ~ class_age

  ))

library(tidyr)

df_periode_finale <- grille_periode %>%
  left_join(df_total_cas, by = c("IRIS", "class_age")) %>%
  mutate(nb_cas_total = replace_na(nb_cas_total, 0)) %>%
  arrange(IRIS, class_age)



df_periode_finale_ordonne <- df_periode_finale %>%
  mutate(
    class_age = factor(class_age, levels = c("[0-14]", "[15-29]", "[30-44]", "[45-59]", "[60-74]", ">=75"))
  ) %>%
  arrange(IRIS, class_age)



cas_total_par_tranche <- df_localisation_age %>%
  group_by(IRIS, class_age) %>%
  summarise(nb_cas_total = n(), .groups = "drop")


df_resume_tranches <- df_periode_finale_ordonne %>%
  group_by(IRIS, class_age) %>%
  summarise(
    REG = first(REG),
    DEP = first(DEP),
    COM = first(COM),
    LIBCOM = first(LIBCOM),
    LIBIRIS = first(LIBIRIS),
    TYP_IRIS = first(TYP_IRIS),
    MODIF_IRIS = first(MODIF_IRIS),
    Population_Totale = sum(Population_Totale, na.rm = TRUE),
    quintile = first(quintile),
    .groups = "drop"
  ) %>%
  arrange(IRIS, class_age)



df_resume_tranches <- df_resume_tranches %>%
  left_join(cas_total_par_tranche, by = c("IRIS", "class_age")) %>%
  mutate(nb_cas_total = replace_na(nb_cas_total, 0))  # au cas où


df_sir_Tcancers <- df_sir_Tcancers %>%
  select(-Population_Totale)




n_distinct(df_periode_finale_ordonne$IRIS)

df_localisation_age %>% distinct(IRIS) %>% count()


##############################################################################

## En attendant les taux d'incidence de LSI 28/07/2025

##############################################################################

### Calcul des SIR BRUT et SUR liss? avec leurs interval de confiance
### Calcul des cas Attendues par tanches d'?ges


library(dplyr)

taux_ref <- Taux_ref %>%
  dplyr::select(Ann?e = ANNEE, class_age, incidence_100k)


taux_ref <- taux_ref %>%
  filter(Ann?e >= 2010 & Ann?e <= 2017)

df_pop_agrege_complet <- df_pop_agrege_complet %>%
  mutate(class_age = recode(class_age,
                            ">=75" = ">=75"))

taux_ref$class_age <- as.character(taux_ref$class_age)


df_cas_attendus <- df_pop_agrege_complet %>%
  left_join(taux_ref, by = c("Annee" = "Ann?e", "class_age")) %>%
  mutate(
    cas_attendus = pop_iris * incidence_100k / 100000
  )

df_cas_attendus <- df_cas_attendus %>%
  select(-IRIS)

df_cas_attendus <- df_cas_attendus %>%
  rename(IRIS = IRIS_corrige)


### ici df_cas_attendus est par ann?e .
### maintenant calculons sur toutes la p?riode




df_attendus_periode <- df_cas_attendus %>%
  group_by(IRIS, class_age) %>%
  summarise(
    pop_totale = sum(pop_iris),
    cas_attendus_total = sum(cas_attendus, na.rm = TRUE),
    .groups = "drop"
  )



df_attendus_periode <- df_cas_attendus %>%
  group_by(IRIS, class_age, REG, DEP, COM, LIBCOM, LIBIRIS, TYP_IRIS, MODIF_IRIS) %>%
  summarise(
    pop_iris = sum(pop_iris, na.rm = TRUE),
    cas_attendus = sum(cas_attendus, na.rm = TRUE),
    .groups = "drop"
  ) %>%
  arrange(IRIS, class_age)



## avec df_sir_final on a les cas_observ? et les cas_attendu qui va nous permettre  de calculer les SIR et IC apr?s


df_sir_final <- df_sir_Tcancers %>%
  left_join(
    df_attendus_periode %>%
      dplyr::select(IRIS, class_age, cas_attendus),
    by = c("IRIS", "class_age")
  )



#################################################################################
## Calculs des SIR avec leurs intervalle de confiance tous cancers 2006-2019

df_sir_final <- df_sir_final %>%
  mutate(
    SIR_brut = if_else(cas_attendus > 0, nb_cas_total / cas_attendus, NA_real_),
    IC_inf_brut = case_when(
      nb_cas_total > 0 ~ qchisq(0.025, 2 * nb_cas_total) / (2 * cas_attendus),
      nb_cas_total == 0 ~ 0
    ),
    IC_sup_brut = case_when(
      nb_cas_total > 0 ~ qchisq(0.975, 2 * (nb_cas_total + 1)) / (2 * cas_attendus),
      nb_cas_total == 0 ~ -log(0.05) / cas_attendus
    )
  )

###############################################################################
######################## Carte 68  ###########################################

library(dplyr)
library(tidyverse)
library(sf)
library(tmap)
library(dplyr)

tmap_mode("view")  # active le mode interactif

df_cartes <- st_read ("C:/Users/KALSACE_M_CONDE00/Desktop/Contour IRIS/2018/CONTOURS-IRIS_2-1__SHP__FRA_2018-01-01/CONTOURS-IRIS/1_DONNEES_LIVRAISON_2018-07-00057/CONTOURS-IRIS_2-1_SHP_LAMB93_FXX-2018/CONTOURS-IRIS.shp")

head(df_cartes)
str(df_cartes)

############## on va filtrer pour obtenir les donn?es du 67 et du 68  #####################

df_cartes_68 <- df_cartes %>%
  filter(substr(INSEE_COM, 1, 2) %in% c("68","67")) ### on doit ajouter 67 pour afficher la carte d'Alsace

# V?rifier l'affich?ge de notre carte 68 mais on peux afficher le 67 aussi

df_cartes_68_modifi? <- df_cartes_68 %>%
  left_join(correspondance_iris, by = c("CODE_IRIS" = "IRIS")) %>%
  mutate(CODE_IRIS = if_else(!is.na(IRIS_nouveau),IRIS_nouveau,CODE_IRIS)) %>%
  select(-IRIS_nouveau)


plot(st_geometry(df_cartes_68_modifi?))


df_cartes_68 <- df_cartes_68 %>%
  left_join(correspondance_iris, by = "IRIS") %>%
  mutate(
    IRIS_corrige = coalesce(CODE_IRIS, IRIS),
    statut_IRIS  = if_else(!is.na(CODE_IRIS), "Fusionn?", "Conserv?")
  ) %>%
  dplyr::select(-CODE_IRIS)

################################################################################
## on va continuer pour afficher la carte des EDI mais j'ai seulement la version de 2011-2015-2017 et la version 2007 n'est pas disponible en t?l?chargement
## il faut faire une demande au niveau de caen

correspondance_iris <- tibble::tribble(
  ~IRIS,        ~IRIS_nouveau,
  "680310000",  "680060000",  # Bernwiller
  "680700000",  "680560201",  # Didenheim → Brunstatt-Didenheim
  "681080000",  "682400000",  # Grentzingen → Illtal
  "681330000",  "682400000",  # Henflingen → Illtal
  "681640000",  "681620000",  # Kientzheim → Kaysersberg-Vignoble
  "682060000",  "680120000",  # Michelbach → Aspach-Michelbach
  "682330000",  "682010000",  # Niederbruck → Masevaux-Niederbruck
  "682720000",  "683630000",  # Riedwihr → Porte-du-Ried
  "683100000",  "681620000",  # Sigolsheim → Kaysersberg-Vignoble
  "683140000",  "681800000",  # Soppe-le-Haut → Le Haut-Soultzbach
  "683190000",  "683200000",   # Spechbach-le-Bas → Spechbach
  "670140000",  "674180000",
  "670410000",  "670040000",
  "671580000",  "675390000",
  "672070000",  "675390000",
  "672970000",  "675390000",
  "673740000",  "674950000",
  "674020000",  "673720000",
  "674310000",  "670040000",
  "674390000",  "672020000",
  "674690000",  "670040000",
  "674960000",  "673720000",
  "675120000",  "673720000",
  "675600000",  "671530000"
)


#############################################################
#### cartes des edi

library(readxl)
edi2011_iris <- read_excel("edi2011-iris (1).xls")
View(edi2011_iris)


library(sf)
library(dplyr)
library(ggplot2)
library(viridis)  # pour une palette de couleurs lisible

#notre shapefile IRIS (d?j? lu dans df_cartes_68)
# la colonne CODE_IRIS doit ?tre au format caract?re

df_cartes_68 <- df_cartes_68_modifi? %>%
  mutate(CODE_IRIS = as.character(CODE_IRIS))

### on va faire une Jointure du shapefile pour chaque version D'EDI

carte_edi2011 <- df_cartes_68 %>%
  left_join(edi2011_iris, by = c("CODE_IRIS" = "IRIS"))


carte_edi2011 <- carte_edi2011 %>%
  mutate(quintileedi2011 = factor(quintileedi2011, levels = 1:5))

palette_edi <- c("#ffffb2", "#fecc5c", "#fd8d3c", "#f03b20", "#bd0026") ## on peux changer la palette de couleur ? tout moment


afficher_carte_quintile <- function(data, colonne_quintile, titre) {
  ggplot(data) +
    geom_sf(aes_string(fill = colonne_quintile), color = "white", size = 0.1) +
    scale_fill_manual(
      values = palette_edi,
      labels = c("1 (moins d?favoris?)", "2", "3", "4", "5 (plus d?favoris?)"),
      drop = TRUE,
      na.value = "grey90"
    ) +
    labs(title = titre, fill = "Quintile EDI") +
    theme_minimal() +
    theme(
      legend.position = "right",
      axis.text = element_blank(),
      axis.ticks = element_blank(),
      panel.grid = element_blank(),
      axis.title = element_blank()
    )
}


carte_edi2011 <- carte_edi2011 %>%
  mutate(quintileedi2011 = factor(quintileedi2011,
                                  levels = 1:5,
                                  labels = c("1(moins d?favoris?)","2","3","4","5 (plus d?favoris?)")))
carte_edi2011$quintileedi2011[carte_edi2011$CODE_IRIS == "680560201"] <- 3

carte_edi2011_filtr?e <- carte_edi2011 %>%
  filter(!is.na(quintileedi2011))

# Carte EDI 2011
afficher_carte_quintile(carte_edi2011_filtr?e, "quintileedi2011", "CARTE EDI 2011 Alsace")

afficher_carte_quintile(carte_edi2011, "quintileedi2011", "Quintiles EDI 2011 Alsace") ## 2?me version avec 1 iris absents


# Affichage Des cartes : c'est bon pour les EDI de 2015-2017


########## Choix de l'edi entre edi 2007-2011
library(dplyr)
library(ggplot2)

df_68 <-df_localisation %>%
  filter(!is.na(quintile),!is.na(A_EDI)) %>%
  mutate(EDI_quintile_harmonise = quintile,
        A_EDI = as.factor(A_EDI))

ggplot(df_68,aes(x = factor(EDI_quintile_harmonise),fill =A_EDI)) +
  geom_bar(position = "dodge") +
  labs(title = "Distribution des EDI quintiles par ann?e Alsace",
       x = "quintile EDI",fill = "Ann?e EDI")


library(dplyr)
library(tidyr)
library(ggplot2)
library(sf)
## Calculs des quintiles moyens par  iris et par ann?e

df_quintiles <-df_localisation %>%
  group_by(IRIS,A_EDI) %>%
  summarise(quintile = mean(quintile,na.rm = TRUE), .groups = "drop") %>%
  pivot_wider(names_from =A_EDI, values_from = quintile,names_prefix = "quintile_") %>%
  mutate(diff_quintile = quintile_2011 - quintile_2007)%>%
  filter(!is.na(diff_quintile))


df_cartes_68_joined <- df_cartes_68 %>%
  left_join(df_quintiles,by = c("CODE_IRIS" = "IRIS"))

palette_edi <- c("#ffffb2", "#fecc5c", "#fd8d3c", "#f03b20", "#bd0026")
##2007
ggplot(df_cartes_68_joined) +
  geom_sf(aes(fill = quintile_2007),color = NA) +
  scale_fill_gradientn(colors = palette_edi,name ="Quintile 2007")+
  labs(title = "Quintiles EDI-Ann?e 2007")+
  theme_void()


##2011
ggplot(df_cartes_68_joined) +
  geom_sf(aes(fill = quintile_2011),color = NA) +
  scale_fill_gradientn(colors = palette_edi,name ="Quintile 2011")+
  labs(title = "Quintiles EDI-Ann?e 2011")+
  theme_void()


summary(df_quintiles$diff_quintile)

df_quintiles <-df_quintiles %>%
  group_by(IRIS,A_EDI) %>%
  summarise(quintile = mean(quintile,na.rm = TRUE), .groups = "drop") %>%
  pivot_wider(names_from =A_EDI, values_from = quintile,names_prefix = "quintile") %>%
  filter (!is.na(quintile_2007) & !is.na(quitile_2011))




df_quintiles <-df_localisation %>%
  group_by(IRIS,A_EDI) %>%
  summarise(quintile = mean(quintile,na.rm = TRUE), .groups = "drop") %>%
  pivot_wider(names_from =A_EDI, values_from = quintile,names_prefix = "quintile_") %>%
  mutate(diff_quintile = quintile_2011 - quintile_2007) %>%
  filter(!is.na(diff_quintile))


summary(df_quintiles$diff_quintile)
df_quintiles %>%
  count(diff_quintile) %>%
  arrange(desc(n))

ggplot(df_quintiles,aes(x = diff_quintile))+
  geom_histogram(binwidth = 1,fill = '#74c476',color = 'white')+
  labs(
    title = "Ecart entre quintile 2011 et 2007",
    x = "Diff?rence de quintile",
    y = "Nombre d'IRIS"

  )+
  theme_minimal()

######################################################################################################
##### Carte int?ractive si on doit manipuler,zoomer ou faire autre chose

tmap_mode("view")

tm_shape(carte_edi2011) +
  tm_fill("quintileedi2011",
          palette = palette_edi,
          style = "cat",
          popup.vars = c(
            "Nom de la commune"     = "NOM_COM",
            "Code INSEE commune"    = "INSEE_COM",
            "Nom IRIS"              = "NOM_IRIS",
            "Code IRIS"             = "CODE_IRIS",
            "Type IRIS"             = "TYP_IRIS.x",
            "Indice EDI 2011"       = "EDI2011",
            "Quintile EDI 2011"     = "quintileedi2011"
          ),
          id = "CODE_IRIS",
          title = "Quintile EDI 2011") +
  tm_borders(lwd = 0.2, col = "white") +
  tm_layout(title = "Carte interactive-Indice EDI 2011 (Haut-Rhin)")

######################################################################################################
######################################################################################################
################ sir pour toute la p?riode 2006-2019- Tous cancers

library(dplyr)
library(sf)
library(ggplot2)

df_sir_final <- df_sir_final %>%
  mutate(IRIS = as.character(IRIS))

df_cartes_68 <- df_cartes_68 %>%
  mutate(CODE_IRIS = as.character(CODE_IRIS))



library(dplyr)

sir_global <- df_sir_final %>%
  group_by(IRIS, LIBCOM, LIBIRIS, TYP_IRIS, MODIF_IRIS) %>%
  summarise(
    nb_cas_total = sum(nb_cas_total, na.rm = TRUE),
    cas_attendus = sum(cas_attendus, na.rm = TRUE),
    .groups = "drop"
  ) %>%
  mutate(
    SIR_brut = if_else(cas_attendus > 0, nb_cas_total / cas_attendus, NA_real_),
    IC_inf = if_else(
      nb_cas_total > 0,
      qchisq(0.025, 2 * nb_cas_total) / (2 * cas_attendus),
      0
    ),
    IC_sup = if_else(
      nb_cas_total > 0,
      qchisq(0.975, 2 * (nb_cas_total + 1)) / (2 * cas_attendus),
      -log(0.05) / cas_attendus
    )
  )



df_sir_map <- df_cartes_68 %>%
  left_join(sir_global, by = c("CODE_IRIS" = "IRIS"))




df_sir_map <- df_sir_map %>%
  left_join(correspondance_iris,by = c("CODE_IRIS" = "IRIS"))

df_sir_map <- df_sir_map %>%
  mutate(CODE_IRIS = if_else(!is.na(IRIS_nouveau),IRIS_nouveau,CODE_IRIS)) %>%
  select(-IRIS_nouveau)

##................................................................................
## on va d?finir les diff?rentes classes des SIR pour faire notre l?gende
## voir s'il n'ya pas une m?thode statistique qui permet de d?finir les classes ???????

df_sir_map$SIR_cat <- dplyr::case_when(
  df_sir_map$SIR_brut == 0 & df_sir_map$cas_attendus < 1 ~ "0 cas (E<1)",
  df_sir_map$SIR_brut == 0 ~ "SIR = 0",
  df_sir_map$SIR_brut < 0.80 ~ "< 0.80",
  df_sir_map$SIR_brut < 0.95 ~ "[0.80 ? 0.95[",
  df_sir_map$SIR_brut < 1.05 ~ "[0.95 ? 1.05[",
  df_sir_map$SIR_brut < 1.20 ~ "[1.05 ? 1.20[",
  df_sir_map$SIR_brut <= 2 ~ "[1.20 ? 2.00]",
  df_sir_map$SIR_brut > 2 ~ "> 2.00"
)

# D?finir les niveaux complets, m?me si certains sont absents

df_sir_map$SIR_cat <- factor(df_sir_map$SIR_cat, levels = c(
  "SIR = 0",
  "< 0.80",
  "[0.80 ? 0.95[",
  "[0.95 ? 1.05[",
  "[1.05 ? 1.20[",
  "[1.20 ? 2.00]",
  "> 2.00"
))

# Palette sans la classe "0 cas (E < 1)"
sir_palette <- c(
  "SIR = 0"           = "#1a9850",  # gris
  "< 0.80"            = "#2166ac",  # bleu fonc?
  "[0.80 ? 0.95["     = "#67a9cf",  # bleu clair
  "[0.95 ? 1.05["     = "#FFFFFF",  # blanc
  "[1.05 ? 1.20["     = "#fdd9a0",  # beige dor? / jaune claire
  "[1.20 ? 2.00]"     = "#fdae61",  # orange doux
  "> 2.00"            = "#d73027"   # ? rouge vif (au lieu de orange fonc?)
)

#  Carte ggplot avec l?gende sans "0 cas (E < 1)" mais avec "> 2.00"

map_sir <-ggplot(df_sir_map) +
  geom_sf(aes(fill = SIR_cat), color = "grey60", size = 0.2) +
  scale_fill_manual(
    values = sir_palette,
    breaks = names(sir_palette),  # force l'ordre complet
    drop = TRUE,                 # garde "> 2.00" même si absente
    na.value = "lightgrey",
    name = "SIR brut"
  ) +
  labs(
    title = "Cartographie des SIR bruts (Tous-Cancers,Alsace)",
    caption = "Le SIR refl?te le rapport entre les cas observ?s et cas attendus dans chaque zone "
  ) +
  coord_sf() +
  theme_void(base_size = 12) +
  theme(
    legend.position = "right",
    axis.text = element_blank(),
    axis.ticks = element_blank(),
    axis.title = element_blank()
  )

#### CARTE Du nombre de cas
library(patchwork)
map_cases <- ggplot(df_sir_map) +
  geom_sf(aes(fill = nb_cas_total),color = "grey60")+
  scale_fill_gradientn(
    colours = c("#FFFFB2","#FECC5C","#FD8D3C","#F03B20","#B10026"),
    name = "Nombre de cas"
  )+
  theme_void() +
  labs(title = "Cartographie du nombre total des cas (Tous cancers,Alsace)" )+
  theme(axis.text = element_blank(),
        axis.ticks = element_blank(),
        axis.title = element_blank())

map_sir+map_cases
map_cases
map_sir ### Carte des SIR_BRUT

library(dplyr)
zone_sans_cas <- df_sir_map %>%
  filter(SIR_brut == 0)
library(openxlsx)
write.csv(zone_sans_cas,"zone avec 0 cas.csv")
#################################################################################################
####### TEST_DE_SIGNIFICATIVITEE sur les SIR_brut

library(dplyr)

df_sir_map_clean <- df_sir_map_clean %>%
  mutate(
    LYSA_cat = case_when(
      SIR_brut == 0 & cas_attendus < 1 ~ "SIR = 0 cas observ?",
      SIR_brut == 0 ~ "SIR = 0 cas observ?",
      IC_sup < 0.95 ~ "Sous-incidence significative",
      IC_inf > 1.05 ~ "Sur-incidence significative",
      TRUE ~ "Non significatif"
    )
  )

lysa_palette <- c(
  "SIR = 0 cas observ?" = "#1a9850",
  "Sous-incidence significative" = "#2166ac",  # bleu
  "Non significatif" = "#FFFFFF",              # bLANC
  "Sur-incidence significative" = "#d73027"    # rouge
)


library(ggplot2)

ggplot(df_sir_map_clean) +
  geom_sf(aes(fill = LYSA_cat), color = "grey60", size = 0.2) +
  scale_fill_manual(
    values = lysa_palette,
    drop = FALSE,
    name = "IC95%(SIR bruts)"
  ) +
  labs(
    title = "Zones de significativit? des SIR bruts (Tous Cancers,Alsace 2010-2017)",
    caption = "Bas? sur l'intervalle de confiance ? 95% IC_sup < 0.95 & IC_inf > 1.05"
  ) +
  coord_sf() +
  theme_void(base_size = 12) +
  theme(
    legend.position = "right",
    plot.title = element_text(face = "bold", size = 14, hjust = 0.5),
    axis.text = element_blank(),
    axis.ticks = element_blank(),
    axis.title = element_blank()
  )

#################################################################################################
################################ M?sure de l'autocorr?lation ####################################
#................................................................................................
#................................................................................................

### Indice de moran et Lisa sur les SIR BRUT pas conseill? car instable mais on peut juste v?rifier

library(sf)
library(spdep)
library(tmap)       # pour la cartographie
library(dplyr)

# Cr?er une liste des polygones voisins
df_sir_map_clean <- df_sir_map %>%
  filter(!is.na(SIR_brut))
set.seed(1234)

nb <- poly2nb(df_sir_map_clean, queen = TRUE)
# Matrice de pond?ration spatiale

lw <- nb2listw(nb, style = "W", zero.policy = TRUE)

moran_test <- moran.test(df_sir_map_clean$SIR_brut, lw, zero.policy = TRUE)
print(moran_test)

############ test de lisa local moran i  #################
##########################################################

install.packages(c("spdep", "sf", "dplyr"))

# Chargement des packages
library(sf)
library(spdep)
library(dplyr)
library(ggplot2)

#  1. Cr?er la matrice de contiguït? (voisinage de type Queen)
nb <- poly2nb(df_sir_map_clean)
listw <- nb2listw(nb, style = "W", zero.policy = TRUE)
set.seed(1234)

#2. Centrer et r?duire la variable SIR_brut
sir_std_vec <- as.numeric(scale(df_sir_map_clean$SIR_brut))  # vecteur num?rique
df_sir_map_clean$SIR_std <- sir_std_vec

# 3. Calcul du Local Moran's I
local_moran <- localmoran(sir_std_vec, listw, zero.policy = TRUE)

# 4. Calcul du lag spatial une seule fois
lag_sir <- lag.listw(listw, sir_std_vec, zero.policy = TRUE)

# 5. Ajouter les r?sultats au data.frame
df_sir_map_clean <- df_sir_map_clean %>%
  mutate(
    Ii = local_moran[, 1],
    p_value = local_moran[, 5],
    lag_sir = lag_sir,
    cluster = case_when(
      SIR_std >= 0 & lag_sir >= 0 & p_value <= 0.05 ~ "High-High",
      SIR_std <= 0 & lag_sir <= 0 & p_value <= 0.05 ~ "Low-Low",
      SIR_std >= 0 & lag_sir <= 0 & p_value <= 0.05 ~ "High-Low",
      SIR_std <= 0 & lag_sir >= 0 & p_value <= 0.05 ~ "Low-High",
      TRUE ~ "Non significatif"
    )
  )

# 6. Carte des clusters LISA
ggplot(df_sir_map_clean) +
  geom_sf(aes(fill = cluster), color = "grey60", size = 0.2) +
  scale_fill_manual(
    values = c(
      "High-High" = "#d73027",     # rouge
      "Low-Low" = "#2166ac",       # bleu
      "High-Low" = "#fdae61",      # orange
      "Low-High" = "#67a9cf",      # bleu clair
      "Non significatif" = "#FFFFFF"
    ),
    name = "Clusters LISA"
  ) +
  labs(
    title = "Carte des clusters LISA-SIR bruts (tous cancers,Alsace 2010-2017)",
    caption = "M?thode : Local Morans I, p ? 0.05"
  ) +
  coord_sf() +
  theme_void(base_size = 12) +
  theme(
    legend.position = "right",
    plot.title = element_text(face = "bold", size = 14, hjust = 0.5),
    axis.text = element_blank(),
    axis.ticks = element_blank(),
    axis.title = element_blank()
  )
# Test de surdispersion avec Dcluster
##############################################################

install.packages("DCluster")
library(DCluster)

# Cr?er un data.frame avec les colonnes n?cessaires
sir_data <- data.frame(
  Observed = df_sir_map_clean$nb_cas_total,
  Expected = df_sir_map_clean$cas_attendus
)
set.seed(1234)
# Test de surdispersion
result <- pottwhitt.test(Observed ~ offset(log(Expected)), data = sir_data, model = "poisson", R = 999)

# Afficher les r?sultats
print(result)

################ Mod?le GLM avec facteur explicative #################
#### je vais utiliser l'edi 2011 pour l'instant ######################

library(dplyr)
carte_edi2011 <- carte_edi2011 %>%
  dplyr::select(-IRIS)

names(carte_edi2011)[names(carte_edi2011) == "CODE_IRIS"] <- "IRIS"


# ?tape 1 : faire le left_join correctement
df_sir_final <- df_sir_final %>%
  left_join(dplyr::select(carte_edi2011,IRIS, EDI2011, quintileedi2011), by = "IRIS")

# ?tape 2 : convertir en facteur uniquement si les colonnes existent
df_sir_final <- df_sir_final %>%
  mutate(
    class_age = as.factor(class_age),
    quintileedi2011 = as.factor(quintileedi2011)
  )

df_sir_final$quintileedi2011 <- recode(df_sir_final$quintileedi2011,
                                       "1(moins d?favoris?)" ="1",
                                       "2" = "2",
                                       "3" = "3",
                                       "4" = "4",
                                       "5 (plus d?favoris?)" = "5")

############ Mod?le Poisson et de surdispersion ########################################
library(dplyr)
set.seed(1234)

glm_model_mc <-df_sir_final %>%
  filter(!is.na(nb_cas_total),
         !is.na(cas_attendus),
         cas_attendus > 0,
         !is.na(quintileedi2011)
)
glm_model <- glm(nb_cas_total ~ + quintileedi2011,
                 offset = log(cas_attendus),
                 family = poisson(link = "log"),
                 data = glm_model_mc)

dispersion <- deviance(glm_model) / df.residual(glm_model)

########################################################################################
############### Mod?le binomial n?gatif ################################################

library(MASS)
set.seed(1234)
glm_nb <- glm.nb(nb_cas_total ~ + quintileedi2011 +
                   offset(log(cas_attendus)),
                 data = glm_model_mc)

AIC(glm_model, glm_nb)

summary(glm_nb)
exp(coef(glm_nb))         # Risques relatifs
exp(confint(glm_nb))      # Intervalles de confiance




# Coefficients exponenti?ls (risques relatifs)

rr <- exp(coef(glm_nb))

# Intervalles de confiance ? 95 %
ci <- exp(confint(glm_nb))

# Cr?er un data frame pour ggplot
df_rr <- data.frame(
  variable = names(rr),
  rr = rr,
  ci_low = ci[, 1],
  ci_high = ci[, 2]
)

library(dplyr)
library(ggplot2)
df_rr_plot2 <- df_rr[-1, ]%>%
  filter(variable %in% paste0("quintiledi2011",2:5))%>%
  mutate(variable = factor(variable,
                           levels =  paste0("quintileid2011",2:5),
                           labels = paste0("EDI-Q", 2:5)))

ggplot(df_rr_plot2, aes(x = variable, y = rr)) +
  geom_hline(yintercept = 1, linetype = "dashed", color = "red") + # -1 pour exclure l'intercept
  geom_point(size = 3, color = "#0072B2") +
  geom_errorbar(aes(ymin = ci_low, ymax = ci_high), width = 0.2, color = "#0072B2") +
  coord_flip() +
  labs(title = "Risques relatifs ajust?s GLM binomial n?gatif (Tous cancers 2010-2017)",
       x = "Variables explicatives",
       y = "Risque relatif (RR)") +
  theme_minimal()

##################################################################################
#################################################################################

######## L? on peux passer au sir liss? ################################
########################################################################

##### sIR lisse ######################################
install.packages("CARBayes")
install.packages("sf")
install.packages("spdep")
install.packages("dplyr")
###############################################################################
#? Packages n?cessaires
library(sf)
library(spdep)
library(CARBayes)
library(dplyr)
library(ggplot2)
library(tmap)
library(patchwork)

# 1. Matrice de voisinage
df_sp <- as(df_sir_map_clean, "Spatial")
neighbors <- poly2nb(df_sp)
W <- nb2mat(neighbors, style = "B", zero.policy = TRUE)

# ? 2. Pr?paration des donn?es
data_model <- df_sir_map_clean %>%
  st_drop_geometry() %>%
  rename(Y = nb_cas_total, E = cas_attendus) %>%
  mutate(region_id = row_number())

set.seed(42)
# ? 3. Mod?e bay?sien de type BYM (CARBayes)
model <- S.CARbym(
  formula = Y ~ offset(log(E)),
  family = "poisson",
  data = data_model,
  W = W,
  burnin = 10000,
  n.sample = 50000,
  thin = 10
)

# 4. R?sultats liss?s
data_model$SIR_liss? <- model$fitted.values / data_model$E
quantiles <- apply(model$samples$fitted, 2, quantile, probs = c(0.025, 0.975))
data_model$IC_lisse_inf <- quantiles[1, ] / data_model$E
data_model$IC_lisse_sup <- quantiles[2, ] / data_model$E

sir_samples <- model$samples$fitted/data_model$E
prob_exces <- colMeans(sir_samples >1.05)
data_model <- data_model %>%
  mutate(SIR_liss? = model$fitted.values/E,
         prob_exces = prob_exces)

# 5. Fusion avec Geom?trie
df_sir_map_lisse <- df_sir_map_clean %>%
  mutate(region_id = row_number())%>%
  left_join(
    dplyr::select(data_model,region_id,SIR_liss?,prob_exces),
    by = "region_id"
  )%>%
  mutate(?toile = if_else(prob_exces > 0.90,"*",""))

# ? 6. Palette partag?e

sir_palette <- c(
  "SIR = 0"           = "#f0f0f0",  # noir
  "< 0.80"            = "#2166ac",  # bleu fonc?
  "[0.80 ? 0.95["     = "#67a9cf",  # bleu clair
  "[0.95 ? 1.05["     = "#FFFFFF",  # gris tr?s clair
  "[1.05 ? 1.20["     = "#fdd9a0",  # beige dor? / jaune claire
  "[1.20 ? 2.00]"     = "#fdae61",  # orange doux
  "> 2.00"            = "#d73027"   # 🔴 rouge vif (au lieu de orange fonc?)
)
#  7. Cat?gorisation harmonis?
cat?goriser_sir <- function(x) {
  case_when(
    is.na(x)        ~ NA_character_,
    x == 0          ~ "0",
    x < 0.80        ~ "< 0.80",
    x < 0.95        ~ "[0.80 ? 0.95[",
    x < 1.05        ~ "[0.95 ? 1.05[",
    x < 1.20        ~ "[1.05 ? 1.20[",
    x <= 2.00       ~ "[1.20 ? 2.00]",
    x > 2.00        ~ "> 2.00"
  )
}

df_sir_map_lisse$SIR_lisse_cat <- factor(cat?goriser_sir(df_sir_map_lisse$SIR_liss?), levels = names(sir_palette))

p_brut <- ggplot(df_sir_map_clean) +
  geom_sf(aes(fill = SIR_cat), color = "grey60", size = 0.2) +
  scale_fill_manual(values = sir_palette, na.value = "lightgrey", name = "SIR brut") +
  labs(title = "SIR bruts par IRIS") +
  theme_void(base_size = 12) +
  theme(legend.position = "none", axis.text = element_blank(),
        axis.ticks = element_blank(), axis.title = element_blank())

p_lisse <- ggplot(df_sir_map_lisse) +
  geom_sf(aes(fill = SIR_lisse_cat), color = "grey60", size = 0.2) +
  geom_sf_text(aes(label = ?toile), size = 4, color = "black") +
  scale_fill_manual(values = sir_palette, na.value = "lightgrey", name = "SIR liss? (BYM)") +
  labs(title = "Carte des SIR_Liss? (Tous cancers Alsace,2010-2017)", caption = "* = P(SIR > 1.05)>0.90") +
  theme_void(base_size = 12) +
  theme(legend.position = "right", axis.text = element_blank(),
        axis.ticks = element_blank(), axis.title = element_blank())

# Affichage
p_brut + p_lisse + plot_layout(ncol = 2)

p_lisse
library(dplyr)
zone_liss? <- df_sir_map_lisse %>%
  filter(?toile == "*")
write.csv2(zone_liss?,"zone 90.csv2")

###### on va zoomer sur les communes

strasbourg <- df_sir_map_lisse %>%
  filter(prob_exces > 0.90)%>%
  group_by(NOM_COM) %>%
  summarise(?toile = n())
print(strasbourg)

library(dplyr)
library( stringr)

df_sir_lisse_strasbourg <- df_sir_map_lisse %>%
  dplyr::filter(str_detect(NOM_COM,"Strasbourg"))

df_sir_lisse_Mulhouse <- df_sir_map_lisse %>%
  dplyr::filter(str_detect(NOM_COM,"Mulhouse"))

df_sir_lisse_saint <- df_sir_map_lisse %>%
  dplyr::filter(str_detect(NOM_COM,"Saint-Louis"))


df_sir_lisse_Illkirch <- df_sir_map_lisse %>%
  dplyr::filter(str_detect(NOM_COM,"Illkirch-Graffenstaden"))

## STRASBOURG
p_Strasbourg <- ggplot(df_sir_lisse_strasbourg) +
  geom_sf(aes(fill = SIR_lisse_cat), color = "grey60", size = 0.2) +
  geom_sf_text(aes(label = ?toile), size = 4, color = "black") +
  scale_fill_manual(values = sir_palette, na.value = "lightgrey", name = "SIR liss? (BYM)") +
  labs(title = "Carte des SIR_Liss? (Tous cancers Strasbourg,2010-2017)", caption = "* = P(SIR > 1.05)>0.90") +
  theme_void(base_size = 12) +
  theme(legend.position = "right", axis.text = element_blank(),
        axis.ticks = element_blank(), axis.title = element_blank())
p_Strasbourg

###Mulhouse

p_Mulhouse <- ggplot(df_sir_lisse_Mulhouse) +
  geom_sf(aes(fill = SIR_lisse_cat), color = "grey60", size = 0.2) +
  geom_sf_text(aes(label = ?toile), size = 4, color = "black") +
  scale_fill_manual(values = sir_palette, na.value = "lightgrey", name = "SIR liss? (BYM)") +
  labs(title = "Carte des SIR_Liss? (Tous cancers Mulhouse,2010-2017)", caption = "* = P(SIR > 1.05)>0.90") +
  theme_void(base_size = 12) +
  theme(legend.position = "right", axis.text = element_blank(),
        axis.ticks = element_blank(), axis.title = element_blank())
p_Mulhouse

###Saint_LOUIS

p_Saint_LOUIS <- ggplot(df_sir_lisse_saint) +
  geom_sf(aes(fill = SIR_lisse_cat), color = "grey60", size = 0.2) +
  geom_sf_text(aes(label = ?toile), size = 4, color = "black") +
  scale_fill_manual(values = sir_palette, na.value = "lightgrey", name = "SIR liss? (BYM)") +
  labs(title = "Carte des SIR_Liss? (Tous cancers Saint-Louis,2010-2017)", caption = "* = P(SIR > 1.05)>0.90") +
  theme_void(base_size = 12) +
  theme(legend.position = "right", axis.text = element_blank(),
        axis.ticks = element_blank(), axis.title = element_blank())
p_Saint_LOUIS



###Illkirch-Graffenstaden

p_Illkirch <- ggplot(df_sir_lisse_Illkirch) +
  geom_sf(aes(fill = SIR_lisse_cat), color = "grey60", size = 0.2) +
  geom_sf_text(aes(label = ?toile), size = 4, color = "black") +
  scale_fill_manual(values = sir_palette, na.value = "lightgrey", name = "SIR liss? (BYM)") +
  labs(title = "Carte des SIR_Liss? (Tous cancers Illkirch-Graffenstaden,2010-2017)", caption = "* = P(SIR > 1.05)>0.90") +
  theme_void(base_size = 12) +
  theme(legend.position = "right", axis.text = element_blank(),
        axis.ticks = element_blank(), axis.title = element_blank())
p_Illkirch


###############################################################################

df_sir_map_lisse <- df_sir_map_clean %>%
  mutate(region_id = row_number())%>%
  left_join(
    dplyr::select(data_model,region_id,SIR_liss?,prob_exces),
    by = "region_id"
  )%>%
  mutate(?toile = if_else(prob_exces > 0.90,"*",""))

df_sir_map_lisse<-df_sir_map_lisse%>%
  dplyr::select(-IRIS)

names(df_sir_map_lisse)[names(df_sir_map_lisse)=="CODE_IRIS"]<- "IRIS"

carte_edi2011$quintileedi2011 <- recode(carte_edi2011$quintileedi2011,
                                       "1(moins d?favoris?)" ="1",
                                       "2" = "2",
                                       "3" = "3",
                                       "4" = "4",
                                       "5 (plus d?favoris?)" = "5")
edi2011<-carte_edi2011%>%
  dplyr::select(IRIS,quintileedi2011)

library(dplyr)

df_sir_map_lisse <- df_sir_map_lisse %>%
  left_join(dplyr::select(carte_edi2011, IRIS, quintileedi2011), by = "IRIS")

df_sir_map_lisse <- st_join(df_sir_map_lisse,edi2011 %>% dplyr::select(IRIS,quintileedi2011))

df_sir_map_lisse<-df_sir_map_lisse %>%
  dplyr::select(-IRIS.x)

#### BYM + EDI

#  1. Chargement des packages
library(sp)
library(spdep)
library(CARBayes)
library(dplyr)
library(sf)
library(ggplot2)

# 2. Conversion en SpatialPolygonsDataFrame (CARBayes ne lit pas les sf)
spatial_data <- as(df_sir_map_lisse, "Spatial")

#  3. Filtrer les IRIS avec donn?es compl?tes
spatial_data_clean <- spatial_data[
  !is.na(spatial_data$nb_cas_total) &
    !is.na(spatial_data$cas_attendus) &
    !is.na(spatial_data$quintileedi2011),
]

#  4. Cr?er l'offset log(E)
spatial_data_clean$log_expected <- log(spatial_data_clean$cas_attendus)

# ? 5. Cr?er la matrice de contiguit? (voisinage)
nb_clean <- poly2nb(spatial_data_clean)
W_clean <- nb2mat(nb_clean, style = "B", zero.policy = TRUE)
set.seed(1234)

# 6. Mod?liser avec CARBayes : mod?le BYM
model_bym <- S.CARbym(
  formula = nb_cas_total ~ quintileedi2011 + offset(log_expected),
  family = "poisson",
  data = spatial_data_clean,
  W = W_clean,
  burnin = 20000,
  n.sample = 100000,
  thin = 10
)

  # 7. Calcul du RR ajust? = lambda / E
spatial_data_clean$RR_bym <- model_bym$fitted.values / spatial_data_clean$cas_attendus
lambda_samples <- model_bym$samples$fitted
sir_samples <- lambda_samples/spatial_data_clean$cas_attendus
prob_exces <- colMeans(sir_samples >1.05)

library(sf)
st_clean <- st_as_sf(spatial_data_clean)
sf_clean <- st_clean %>%
  mutate(prob_exces = prob_exces,
         etoile = if_else(prob_exces > 0.90 ,"*","")
         )

# 8. Repasser en sf pour ggplot
spatial_sf <- st_as_sf(sf_clean)


#  9. Visualiser les RR BYM
ggplot(spatial_sf) +
  geom_sf(aes(fill = SIR_cat), color = "grey60", size = 0.1) +
  geom_sf_text(aes(label = etoile), size = 4, color = "black")+
  scale_fill_manual(
    values = c(
      "< 0.80" = "#2171b5",
      "[0.80 ? 0.95[" ="#c6dbef",
      "[0.95 ? 1.05[" = "#FFFFFF",
      "[1.05 ? 1.20[" = "#fdd9a0",
      "[1.20 ? 2.00]"= "#fdae61",
      "> 2" = "#d73027"
    ),
    name = "RR BYM"
  ) +
  theme_void() +
  labs(
    title = "Risques relatifs (Tous-cancers,2010-2017)",
    subtitle = "Effet spatial structur? + quintiles EDI2011",
    caption = "* P(RR>1.05)> 0.90"
  )
rr <- spatial_sf %>%
  filter(etoile == "*")

library(ggplot2)

ggplot(spatial_sf, aes(x = factor(quintileedi2011), y = RR_bym)) +
  geom_boxplot(fill = "#de2d26", color = "black", outlier.color = "gray30") +
  labs(
    title = "Distribution des risques relatifs BYM selon le quintile EDI Tous-cancers(2010-2017)",
    x = "Quintile EDI (1 = favoris?, 5 = d?favoris?)",
    y = "RR ajust? (??i/Ei)"
  ) +
  theme_minimal()


model_bym
rr
library(writexl)
write_xlsx(rr,"rr.xlsx")
write_xlsx(zone_sans_cas,"Zone avec 0 cas.xlsx")
write_xlsx(zone_liss?,"zone liss? p90.xlsx")
###############################################################################
######################################################
######## SATSCAN  ### on doit installer d'abord sur le pc donc j'attends la confirmation

################################################################################
################################################################################
################################################################################
####### Analyse par type de cancers

# on va r?grouper tous les taux de r?f?rence dans une seule base Taux_ref_total :

Taux_ref_total <- bind_rows(
    Sein_F_ %>% mutate(Localisation = "Sein (F)"),
    Poumon %>% mutate(Localisation = "Poumon"),
    Prostate_H_ %>% mutate(Localisation = "Prostate (H)"),
    Melanomes %>% mutate(Localisation = "Melanomes")
  )


Taux_ref_total <- Taux_ref_total %>%
  filter(ANNEE >= 2010 & ANNEE <= 2017)

write.csv(Taux_ref_total,"Taux_ref_total.csv")   ## ici on a les taux_incidence de r?ference pour chaque type de cancer


Taux_ref_total <- Taux_ref_total %>%
  mutate(Localisation = recode(Localisation, "Melanomes" = "M?lanome cutan?"))

df_localisation_age <- df_localisation_age %>%
  mutate(
    topo_final = case_when(
      topo_final == "Sein"     & i_sexe == 2 ~ "Sein (F)",
      topo_final == "Prostate" & i_sexe == 1 ~ "Prostate (H)",
      TRUE ~ topo_final  # on garde les autres inchang?s
    )
  )

types_cancers <- c("Sein (F)", "Poumon", "Prostate (H)", "M?lanome cutan?")  ### localisation ? analyser mais on peut augmenter

df_pop_corrige_68 <- df_pop_long_corrige_sexe %>%
  filter(DEP %in% c("67","68"))
df_pop_corrige_68 <- df_pop_corrige_68 %>%
  rename(CODE_IRIS = IRIS_corrige)
df_pop_corrige_68 <- df_pop_corrige_68 %>%
  dplyr::select(-IRIS)
df_pop_corrige_68 <- df_pop_corrige_68 %>%
  rename(IRIS= CODE_IRIS)

df_localisation %>%
  group_by(topo_final) %>%
  summarise(total_cas = sum(is_cas,na.rm = TRUE)) %>%
  arrange(desc(total_cas))
### Fonction g?n?rale pour calculer les cas observ?s et attendus pour chaque types de cancers et par tranches ?ges
##################################################################################################################
library(tidyr)
library(dplyr)
library(purrr)

df_detail_age_cancer <- map_dfr(types_cancers, function(cancer) {

  message(" Traitement : ", cancer)

  # Sexe attendu
  sexe_cible <- case_when(
    cancer == "Prostate (H)" ~ "Homme",
    cancer == "Sein (F)"     ~ "Femme",
    TRUE                     ~ NA_character_
  )

  # 1. Taux de r?f?rence
  taux_cancer <- Taux_ref_total %>%
    filter(Localisation == cancer) %>%
    { if (!is.na(sexe_cible)) filter(., SEXE == sexe_cible) else . } %>%
    dplyr::select(ANNEE, class_age, incidence_100k)

  # ? 2. Cas individuels
  cas_cancer <- df_localisation_age %>%
    filter(topo_final == cancer) %>%
    rename(Annee = Ann?e)

  #  3. Cas observ?s
  cas_obs <- cas_cancer %>%
    distinct(IDTUM_KEA, .keep_all = TRUE) %>%
    group_by(IRIS, Annee, class_age) %>%
    summarise(nb_cas_total = n(), .groups = "drop")

  #  4. Population selon le cancer
  pop_cancer <- if (!is.na(sexe_cible)) {
    df_pop_corrige_68 %>%
      filter(sexe == sexe_cible)
  } else {
    df_pop_corrige_68 %>%
      group_by(IRIS, Annee, class_age, REG, DEP, COM, LIBCOM, LIBIRIS,
               TYP_IRIS, MODIF_IRIS, statut_IRIS) %>%
      summarise(
        pop_sexe = sum(pop_sexe),
        pop_iris = first(pop_iris),
        .groups = "drop"
      )
  }

  #  5. Cas attendus
  cas_att <- pop_cancer %>%
    left_join(taux_cancer, by = c("Annee" = "ANNEE", "class_age")) %>%
    mutate(
      incidence_100k = replace_na(incidence_100k, 0),
      cas_attendus = pop_sexe * incidence_100k / 100000
    ) %>%
    group_by(IRIS, Annee, class_age) %>%
    summarise(cas_attendus = sum(cas_attendus), .groups = "drop")

  # 6. Jointure obs + att
  df_tmp <- full_join(cas_obs, cas_att, by = c("IRIS", "Annee", "class_age")) %>%
    mutate(
      nb_cas_total = replace_na(nb_cas_total, 0),
      cas_attendus = replace_na(cas_attendus, 0),
      type_cancer = cancer
    )

  # 7. Infos IRIS
  infos_iris <- df_pop_corrige_68 %>%
    dplyr::select(IRIS, Annee, class_age, REG, DEP, COM, LIBCOM, LIBIRIS,
                  TYP_IRIS, MODIF_IRIS, statut_IRIS, pop_iris) %>%
    distinct()

  # 8. Fusion finale
  df_tmp %>%
    left_join(infos_iris, by = c("IRIS", "Annee", "class_age"))
})

###################################################################################
###################################################################################
###################################################################################
################################################################################
################################################################################
#### Calcul des SIR

### SIR BRUT
library(dplyr)
library(sf)
library(tmap)

# 1. Harmoniser les identifiants
df_detail_age_cancer <- df_detail_age_cancer %>%
  mutate(IRIS = as.character(IRIS))


df_detail_age_cancer <- df_detail_age_cancer %>%
  filter(Annee >= 2010 & Annee <= 2017)

df_cartes_68 <- df_cartes_68 %>%
  mutate(CODE_IRIS = as.character(CODE_IRIS))

# 2. Agr?ger les cas observ?s et attendus par IRIS × type de cancer
sir_par_cancer <- df_detail_age_cancer %>%
  group_by(IRIS, LIBCOM, LIBIRIS, TYP_IRIS, MODIF_IRIS, type_cancer) %>%
  summarise(
    nb_cas_total = sum(nb_cas_total, na.rm = TRUE),
    cas_attendus = sum(cas_attendus, na.rm = TRUE),
    .groups = "drop"
  ) %>%
  mutate(
    SIR_brut = if_else(cas_attendus > 0, nb_cas_total / cas_attendus, NA_real_),
    IC_inf = case_when(
      nb_cas_total > 0 & cas_attendus > 0 ~ qchisq(0.025, 2 * nb_cas_total) / (2 * cas_attendus),
      nb_cas_total == 0 & cas_attendus > 0 ~ 0,
      TRUE ~ NA_real_
    ),
    IC_sup = case_when(
      nb_cas_total > 0 & cas_attendus > 0 ~ qchisq(0.975, 2 * (nb_cas_total + 1)) / (2 * cas_attendus),
      nb_cas_total == 0 & cas_attendus > 0 ~ qchisq(0.975, 2) / (2 * cas_attendus),
      TRUE ~ NA_real_
    )

 )

# 3. Jointure avec les polygones IRIS
sir_map_cancer <- df_cartes_68 %>%
  left_join(sir_par_cancer, by = c("CODE_IRIS" = "IRIS"))


library(dplyr)

# 1. Partir de sir_map_cancer et agr?ger par IRIS × type de cancer
sir_map_cancer_agr?g? <- sir_map_cancer %>%
  group_by(CODE_IRIS, type_cancer) %>%
  summarise(
    nb_cas_total = sum(nb_cas_total, na.rm = TRUE),
    cas_attendus = sum(cas_attendus, na.rm = TRUE),
    geometry = first(geometry),  # conserver la Geom?trie unique
    LIBCOM = first(LIBCOM),
    LIBIRIS = first(LIBIRIS),
    TYP_IRIS.x = first(TYP_IRIS.x),
    MODIF_IRIS = first(MODIF_IRIS),
    .groups = "drop"
  ) %>%
  mutate(
    SIR_brut = if_else(cas_attendus > 0, nb_cas_total / cas_attendus, NA_real_),
    IC_inf = case_when(
      nb_cas_total > 0 & cas_attendus > 0 ~ qchisq(0.025, 2 * nb_cas_total) / (2 * cas_attendus),
      nb_cas_total == 0 & cas_attendus > 0 ~ 0,
      TRUE ~ NA_real_
    ),
    IC_sup = case_when(
      nb_cas_total > 0 & cas_attendus > 0 ~ qchisq(0.975, 2 * (nb_cas_total + 1)) / (2 * cas_attendus),
      nb_cas_total == 0 & cas_attendus > 0 ~ qchisq(0.975, 2) / (2 * cas_attendus),
      TRUE ~ NA_real_
    )

  )

###################################################################################


# D?finir les niveaux complets, m?me si certains sont absents

# Palette sans la classe "0 cas (E < 1)"
sir_palette <- c(
  "SIR = 0"           = "#1a9850",  # gris
  "< 0.80"            = "#2166ac",  # bleu fonc?
  "[0.80 ? 0.95["     = "#67a9cf",  # bleu clair
  "[0.95 ? 1.05["     = "#FFFFFF",  # blanc
  "[1.05 ? 1.20["     = "#fdd9a0",  # beige dor? / jaune claire
  "[1.20 ? 2.00]"     = "#fdae61",  # orange doux
  "> 2.00"            = "#d73027"   # ? rouge vif (au lieu de orange fonc?)
)

###################################################################################

sir_map_cancer_agr?g? <- sir_map_cancer_agr?g? %>%
  mutate(
    SIR_cat = case_when(
      is.na(SIR_brut)     ~ "donn?es manquantes",
      SIR_brut == 0       ~  "SIR = 0",
      SIR_brut < 0.80     ~ "< 0.80",
      SIR_brut < 0.95     ~ "[0.80 ? 0.95[",
      SIR_brut < 1.05     ~ "[0.95 ? 1.05[",
      SIR_brut < 1.20     ~ "[1.05 ? 1.20[",
      SIR_brut <= 2.00    ~ "[1.20 ? 2.00]",
      SIR_brut > 2.00     ~ "> 2.00"
      )
  )
sir_map_cancer_agr?g? <- sir_map_cancer_agr?g? [!is.na(sir_map_cancer_agr?g?$SIR_cat),]
sir_map_cancer_agr?g?$SIR_cat <- factor(sir_map_cancer_agr?g?$SIR_cat, levels = c(
  "SIR = 0",
  "< 0.80",
  "[0.80 ? 0.95[",
  "[0.95 ? 1.05[",
  "[1.05 ? 1.20[",
  "[1.20 ? 2.00]",
  "> 2.00"
))

sir_map_cancer_agr?g?[is.na(sir_map_cancer_agr?g?$SIR_cat),]

library(tmap)
tmap_mode("view")

tm_shape(sir_map_cancer_agr?g?) +
  tm_polygons(
    col           = "SIR_cat",
    palette       = sir_palette,
    title         = "SIR brut",
    drop.levels   = TRUE,
    textNA        = "Donn?e manquante",
    colorNA       = "lightgrey",
    id            = "LIBIRIS",
    popup.vars    = c(
      "Cancer"      = "type_cancer",
      "Commune"     = "LIBCOM",
      "Nom IRIS"    = "LIBIRIS",
      "SIR brut"    = "SIR_brut",
      "IC95 inf."   = "IC_inf",
      "IC95 sup."   = "IC_sup",
      "Obs."        = "nb_cas_total",
      "Attendus"    = "cas_attendus"
    )
  ) +
  tm_facets(by = "type_cancer") +
  tm_layout(
    title             = "SIR brut par type de cancer",
    legend.outside    = TRUE,
    legend.title.size = 1.1
  )



library(ggplot2)

ggplot(sir_map_cancer_agr?g? ) +
  geom_sf(aes(fill = SIR_cat), color = "grey60", size = 0.2) +
  facet_wrap(~ type_cancer) +
  scale_fill_manual(
    values    = sir_palette,
    na.value  = "lightgrey",
    name      = "SIR brut",
    drop      = TRUE
  ) +
  labs(
    title    = "SIR brut par type de cancer",
    caption  = "Cartes bas?es sur le SIR brut"
  ) +
  theme_void() +
  theme(
    legend.position = "right",
    axis.text       = element_blank(),
    axis.ticks      = element_blank(),
    axis.title      = element_blank(),
    strip.text      = element_text(face = "bold", size = 11)
  )



library(ggplot2)
library(dplyr)
data_poumon <- sir_map_cancer_agr?g? %>% filter(type_cancer == "Poumon")
data_Sein <- sir_map_cancer_agr?g? %>% filter(type_cancer == "Sein (F)")
data_Prosate  <- sir_map_cancer_agr?g? %>% filter(type_cancer == "Prostate (H)")
data_m?lanome <- sir_map_cancer_agr?g? %>% filter(type_cancer == "M?lanome cutan?")

types_cancer <- unique(data_m?lanome$type_cancer) ### on remplace par le type de cancer qu'on veux afficher

for (cancer in types_cancer) {
  data_cancer <- sir_map_cancer_agr?g? %>% filter(type_cancer == cancer)

  p <- ggplot(data_cancer) +
    geom_sf(aes(fill = SIR_cat), color = "grey60", size = 0.2) +
    scale_fill_manual(
      values    = sir_palette,
      na.value  = "lightgrey",
      name      = "SIR brut",
      drop      = TRUE
    ) +
    labs(
      title    = paste( "SIR bruts -",cancer,"Alsace,2010-2017"),
      caption  = "Cartes bas?es sur le SIR brut"
    ) +
    theme_void() +
    theme(
      legend.position = "right",
      axis.text       = element_blank(),
      axis.ticks      = element_blank(),
      axis.title      = element_blank(),
      strip.text      = element_text(face = "bold", size = 11),
      plot.title      = element_text(face = "bold",size =  14,hjust = 0.5)
    )

}

print(p)


#Carte d'incidence

library(ggplot2)

cas_poumon <- sir_map_cancer_agr?g? %>% filter(type_cancer == "Poumon")
cas_Sein <- sir_map_cancer_agr?g? %>% filter(type_cancer == "Sein (F)")
cas_Prosate  <- sir_map_cancer_agr?g? %>% filter(type_cancer == "Prostate (H)")
cas_m?lanome <- sir_map_cancer_agr?g? %>% filter(type_cancer == "M?lanome cutan?")

##### CARTE DU NOMBE DE CAS POUMON

ggplot(cas_poumon) +
  geom_sf(aes(fill = nb_cas_total), color = "grey60", size = 0.2) +
  scale_fill_gradientn(
    colours = c("#FFFFB2","#FECC5C","#FD8D3C","#F03B20","#BD0026"),
    name = "Nombre de cas",
    na.value = "lightgrey"
  ) +
  facet_wrap(~ type_cancer) +
  labs(
    title    = "Cartographie des cas observ?s de cancer en Alsace - Poumon",
    caption  = "bas? sur le nombre total des cas observ?s"
  ) +
  theme_void() +
  theme(
    legend.position = "right",
    axis.text       = element_blank(),
    axis.ticks      = element_blank(),
    axis.title      = element_blank(),
    strip.text      = element_text(face = "bold", size = 11)
  )

##### CARTE DU NOMBE DE CAS SEIN (F)

ggplot(cas_Sein) +
  geom_sf(aes(fill = nb_cas_total), color = "grey60", size = 0.2) +
  scale_fill_gradientn(
    colours = c("#FFFFB2","#FECC5C","#FD8D3C","#F03B20","#BD0026"),
    name = "Nombre de cas",
    na.value = "lightgrey"
  ) +
  facet_wrap(~ type_cancer) +
  labs(
    title    = "Cartographie des cas observ?s de cancer en Alsace - Sein (F)",
    caption  = "bas? sur le nombre total des cas observ?s"
  ) +
  theme_void() +
  theme(
    legend.position = "right",
    axis.text       = element_blank(),
    axis.ticks      = element_blank(),
    axis.title      = element_blank(),
    strip.text      = element_text(face = "bold", size = 11)
  )

##### CARTE DU NOMBE DE CAS PROSTATE (H)

ggplot(cas_Prosate) +
  geom_sf(aes(fill = nb_cas_total), color = "grey60", size = 0.2) +
  scale_fill_gradientn(
    colours = c("#FFFFB2","#FECC5C","#FD8D3C","#F03B20","#BD0026"),
    name = "Nombre de cas",
    na.value = "lightgrey"
  ) +
  facet_wrap(~ type_cancer) +
  labs(
    title    = "Cartographie des cas observ?s de cancer en Alsace - Prostate (H)",
    caption  = "bas? sur le nombre total des cas observ?s"
  ) +
  theme_void() +
  theme(
    legend.position = "right",
    axis.text       = element_blank(),
    axis.ticks      = element_blank(),
    axis.title      = element_blank(),
    strip.text      = element_text(face = "bold", size = 11)
  )

##### CARTE DU NOMBE DE CAS MELANOME

ggplot(cas_m?lanome) +
  geom_sf(aes(fill = nb_cas_total), color = "grey60", size = 0.2) +
  scale_fill_gradientn(
    colours = c("#FFFFB2","#FECC5C","#FD8D3C","#F03B20","#BD0026"),
    name = "Nombre de cas",
    na.value = "lightgrey"
  ) +
  facet_wrap(~ type_cancer) +
  labs(
    title    = "Cartographie des cas observ?s de cancer en Alsace - M?lanome cutan?",
    caption  = "bas? sur le nombre total des cas observ?s"
  ) +
  theme_void() +
  theme(
    legend.position = "right",
    axis.text       = element_blank(),
    axis.ticks      = element_blank(),
    axis.title      = element_blank(),
    strip.text      = element_text(face = "bold", size = 11)
  )


  #### Indice de Moran Pour chaque types de cancers


library(dplyr)
df_detail_age_cancer <- df_detail_age_cancer %>%
  dplyr::filter(!is.na(LIBCOM))

sapply(df_detail_age_cancer,function(x) sum(is.na(x)))

geom_iris <- sir_map_cancer_agr?g? %>%
  dplyr::distinct(CODE_IRIS, .keep_all = TRUE) %>%
  dplyr::select(CODE_IRIS,geometry)

library(spdep)

results_moran <- list()

for(cancer in unique(df_detail_age_cancer$type_cancer)) {
  cat("Test de Moran pour cancer :", cancer, "\n")

  df_sub <- df_detail_age_cancer %>%
    filter(type_cancer == cancer) %>%
    left_join(geom_iris, by = c("IRIS" = "CODE_IRIS")) %>%
    st_as_sf() %>%
    group_by(IRIS) %>%
    summarise(nb_cas_total = sum(nb_cas_total, na.rm = TRUE),
              geometry = first(geometry)) %>%
    ungroup() %>%
    st_as_sf()

  nb <- poly2nb(df_sub, queen = TRUE)
  lw <- nb2listw(nb, style = "W", zero.policy = TRUE)
  set.seed(1234)

  moran_res <- moran.test(df_sub$nb_cas_total, lw, zero.policy = TRUE)

  results_moran[[cancer]] <- moran_res

  print(moran_res)
}

#######################################################################
# Fonction personnalis?e de surdispersion

test_surdispersion <- function(obs, exp) {
  ratio <- obs / exp
  var(ratio) / mean(ratio)^2
}

# Fonction de surdispersion : coefficient de variation des SIR
test_surdispersion <- function(obs, exp) {
  sir <- obs / exp
  cv2 <- var(sir, na.rm = TRUE) / mean(sir, na.rm = TRUE)^2
  return(cv2)
}
# Application à chaque type de cancer
sir_map_cancer_agr?g? %>%
  st_drop_geometry() %>%
  group_by(type_cancer) %>%
  summarise(
    stat_surdispersion = test_surdispersion(nb_cas_total, cas_attendus),
    .groups = "drop"
  ) %>%
  arrange(desc(stat_surdispersion))



# Application par type de cancer
df_surdispersion <- sir_map_cancer_agr?g? %>%
  st_drop_geometry() %>%
  group_by(type_cancer) %>%
  summarise(
    stat_surdispersion = test_surdispersion(nb_cas_total, cas_attendus),
    .groups = "drop"
  ) %>%
  arrange(desc(stat_surdispersion))

library(DCluster)
pottwhitt.test(Observed ~ offset(log(Expected)), data = sir_data_lisse, model = "poisson", R = 999)


## carte lisa

  library(spdep)
  library(dplyr)
  library(sf)

  # Initialiser une liste pour stocker les r?sultats
  lisa_results <- list()

  # Liste des types de cancer
  types_cancer <- unique(sir_map_cancer_agr?g?$type_cancer)

  # Boucle sur chaque type de cancer
  for (cancer in types_cancer) {
    message("Traitement : ", cancer)

    # Filtrer les donn?es pour ce type de cancer
    sf_cancer <- sir_map_cancer_agr?g? %>%
      filter(type_cancer == cancer & !is.na(SIR_brut))

    # Cr?er les voisins et poids spatiaux
    nb <- poly2nb(sf_cancer, queen = TRUE)
    lw <- nb2listw(nb, style = "W", zero.policy = TRUE)

    # Calcul du Moran local
    moran <- localmoran(sf_cancer$SIR_brut, lw, zero.policy = TRUE)

    # Ajouter les r?sultats au tableau
    sf_cancer <- sf_cancer %>%
      mutate(
        local_I     = moran[, 1],
        local_p     = moran[, 5],
        cluster_sig = case_when(
          SIR_brut > mean(SIR_brut, na.rm = TRUE) & local_I > 0 & local_p < 0.05 ~ "High-High",
          SIR_brut < mean(SIR_brut, na.rm = TRUE) & local_I > 0 & local_p < 0.05 ~ "Low-Low",
          SIR_brut > mean(SIR_brut, na.rm = TRUE) & local_I < 0 & local_p < 0.05 ~ "High-Low",
          SIR_brut < mean(SIR_brut, na.rm = TRUE) & local_I < 0 & local_p < 0.05 ~ "Low-High",
          TRUE ~ "Non significatif"
        )
      )

    # Stocker le r?sultat
    lisa_results[[cancer]] <- sf_cancer
  }

  # Fusionner tous les r?sultats
  sf_lisa_all <- do.call(rbind, lisa_results)

  lisa_poumon <- sf_lisa_all %>% filter(type_cancer == "Poumon")
  lisa_Sein <- sf_lisa_all %>% filter(type_cancer == "Sein (F)")
  lisa_Prosate  <- sf_lisa_all %>% filter(type_cancer == "Prostate (H)")
  lisa_m?lanome <- sf_lisa_all %>% filter(type_cancer == "M?lanome cutan?")


  ##### CARTE LISA POUMON

  library(ggplot2)

  ggplot(lisa_poumon) +
    geom_sf(aes(fill = cluster_sig), color = "grey60", size = 0.2) +
    facet_wrap(~ type_cancer) +
    scale_fill_manual(
      values = c(
        "High-High"        = "#d73027",
        "Low-Low"          = "#4575b4",
        "High-Low"         = "#fdae61",
        "Low-High"         = "#abd9e9",
        "Non significatif" = "lightgrey"
      ),
      name = "Cluster Local",
      drop = FALSE
    ) +
    labs(
      title   = "Clusters significatifs (Moran local) Poumon",
      caption = "p < 0.05"
    ) +
    theme_void() +
    theme(
      legend.position = "right",
      axis.text       = element_blank(),
      axis.ticks      = element_blank(),
      axis.title      = element_blank(),
      strip.text      = element_text(face = "bold", size = 11)
    )

  ##### CARTE LISA SEIN (F)

  library(ggplot2)

  ggplot(lisa_Sein) +
    geom_sf(aes(fill = cluster_sig), color = "grey60", size = 0.2) +
    facet_wrap(~ type_cancer) +
    scale_fill_manual(
      values = c(
        "High-High"        = "#d73027",
        "Low-Low"          = "#4575b4",
        "High-Low"         = "#fdae61",
        "Low-High"         = "#abd9e9",
        "Non significatif" = "lightgrey"
      ),
      name = "Cluster Local",
      drop = FALSE
    ) +
    labs(
      title   = "Clusters significatifs (Moran local) Sein (F)",
      caption = "p < 0.05"
    ) +
    theme_void() +
    theme(
      legend.position = "right",
      axis.text       = element_blank(),
      axis.ticks      = element_blank(),
      axis.title      = element_blank(),
      strip.text      = element_text(face = "bold", size = 11)
    )

  ##### CARTE LISA PROSTATE (H)

  library(ggplot2)

  ggplot(lisa_Prosate) +
    geom_sf(aes(fill = cluster_sig), color = "grey60", size = 0.2) +
    facet_wrap(~ type_cancer) +
    scale_fill_manual(
      values = c(
        "High-High"        = "#d73027",
        "Low-Low"          = "#4575b4",
        "High-Low"         = "#fdae61",
        "Low-High"         = "#abd9e9",
        "Non significatif" = "lightgrey"
      ),
      name = "Cluster Local",
      drop = FALSE
    ) +
    labs(
      title   = "Clusters significatifs (Moran local) Prostate(H)",
      caption = "p < 0.05"
    ) +
    theme_void() +
    theme(
      legend.position = "right",
      axis.text       = element_blank(),
      axis.ticks      = element_blank(),
      axis.title      = element_blank(),
      strip.text      = element_text(face = "bold", size = 11)
    )

  ##### CARTE LISA MELANOME

  library(ggplot2)

  ggplot(lisa_m?lanome) +
    geom_sf(aes(fill = cluster_sig), color = "grey60", size = 0.2) +
    facet_wrap(~ type_cancer) +
    scale_fill_manual(
      values = c(
        "High-High"        = "#d73027",
        "Low-Low"          = "#4575b4",
        "High-Low"         = "#fdae61",
        "Low-High"         = "#abd9e9",
        "Non significatif" = "lightgrey"
      ),
      name = "Cluster Local",
      drop = FALSE
    ) +
    labs(
      title   = "Clusters significatifs (Moran local) M?lanome cutan?",
      caption = "p < 0.05"
    ) +
    theme_void() +
    theme(
      legend.position = "right",
      axis.text       = element_blank(),
      axis.ticks      = element_blank(),
      axis.title      = element_blank(),
      strip.text      = element_text(face = "bold", size = 11)
    )


  ##### ZONE DE SIGNIFICATIVE
  library(dplyr)

  sir_map_cancer_agr?g? <- sir_map_cancer_agr?g? %>%
    mutate(
      LYSA_cat = case_when(
        SIR_brut == 0 & cas_attendus < 1 ~ "0 cas (SIR = 0 cas observ?",
        SIR_brut == 0 ~ "SIR = 0 cas observ?",
        IC_sup < 0.95 ~ "Sous-incidence significative",
        IC_inf > 1.05 ~ "Sur-incidence significative",
        TRUE ~ "Non significatif"
      )
    )


  library(dplyr)

  df_sir_map_clean <- df_sir_map_clean %>%
    mutate(
      LYSA_cat = case_when(
        SIR_brut == 0 & cas_attendus < 1 ~ "SIR = 0 cas observ?",
        SIR_brut == 0 ~ "SIR = 0 cas observ?",
        IC_sup < 0.95 ~ "Sous-incidence significative",
        IC_inf > 1.05 ~ "Sur-incidence significative",
        TRUE ~ "Non significatif"
      )
    )

  lysa_palette <- c(
    "SIR = 0 cas observ?" = "#1a9850",
    "Sous-incidence significative" = "#2166ac",  # bleu
    "Non significatif" = "#FFFFFF",              # bLANC
    "Sur-incidence significative" = "#d73027"    # rouge
  )

  library(ggplot2)

  ggplot(sir_map_cancer_agr?g?) +
    geom_sf(aes(fill = LYSA_cat), color = "grey60", size = 0.2) +
    facet_wrap(~ type_cancer) +
    scale_fill_manual(
      values = lysa_palette,
      drop = FALSE,
      name = "IC95%(SIR_BRUT)"
    ) +
    labs(
      title = "Zones de sur- ou sous-incidence significative selon le SIR_BRUT",
      subtitle = "Par type de cancer ? Haut-Rhin (2006-2019)",
      caption = "Bas? sur l'intervalle de confiance ? 95% IC_sup <0.95 & IC_inf >1.05"
    ) +
    coord_sf() +
    theme_void(base_size = 12) +
    theme(
      legend.position = "right",
      plot.title = element_text(face = "bold", size = 14, hjust = 0.5),
      plot.subtitle = element_text(size = 12, hjust = 0.5),
      axis.text = element_blank(),
      axis.ticks = element_blank(),
      axis.title = element_blank(),
      strip.text = element_text(face = "bold", size = 11)
    )

#######################################################################################


  p_poumon <- sir_map_cancer_agr?g? %>% filter(type_cancer == "Poumon")
  p_Sein <- sir_map_cancer_agr?g? %>% filter(type_cancer == "Sein (F)")
  p_Prosate  <- sir_map_cancer_agr?g? %>% filter(type_cancer == "Prostate (H)")
  p_m?lanome <- sir_map_cancer_agr?g? %>% filter(type_cancer == "M?lanome cutan?")

  ##### CARTE SIGNIFICATIVE POUMON

  library(ggplot2)

  ggplot(p_poumon) +
    geom_sf(aes(fill = LYSA_cat), color = "grey60", size = 0.2) +
    facet_wrap(~ type_cancer) +
    scale_fill_manual(
      values = lysa_palette,
      drop = FALSE,
      name = "IC95%(SIR_BRUT)"
    ) +
    labs(
      title = "Zones de sur- ou sous-incidence significative selon le SIR_BRUT",
      subtitle = "Cancer du Poumon 2010-2017",
      caption = "Bas? sur l'intervalle de confiance ? 95% IC_sup <0.95 & IC_inf >1.05"
    ) +
    coord_sf() +
    theme_void(base_size = 12) +
    theme(
      legend.position = "right",
      plot.title = element_text(face = "bold", size = 14, hjust = 0.5),
      plot.subtitle = element_text(size = 12, hjust = 0.5),
      axis.text = element_blank(),
      axis.ticks = element_blank(),
      axis.title = element_blank(),
      strip.text = element_text(face = "bold", size = 11)
    )

  ##### CARTE SIGNIFICATIVE PROSTATE (H)

  library(ggplot2)

  ggplot(p_Prosate) +
    geom_sf(aes(fill = LYSA_cat), color = "grey60", size = 0.2) +
    facet_wrap(~ type_cancer) +
    scale_fill_manual(
      values = lysa_palette,
      drop = FALSE,
      name = "IC95%(SIR_BRUT)"
    ) +
    labs(
      title = "Zones de sur- ou sous-incidence significative selon le SIR_BRUT",
      subtitle = "Cancer de la prostate (H) 2010-2017",
      caption = "Bas? sur l'intervalle de confiance ? 95% IC_sup <0.95 & IC_inf >1.05"
    ) +
    coord_sf() +
    theme_void(base_size = 12) +
    theme(
      legend.position = "right",
      plot.title = element_text(face = "bold", size = 14, hjust = 0.5),
      plot.subtitle = element_text(size = 12, hjust = 0.5),
      axis.text = element_blank(),
      axis.ticks = element_blank(),
      axis.title = element_blank(),
      strip.text = element_text(face = "bold", size = 11)
    )

  ##### CARTE SIGNIFICATIVE SEIN(F)

  library(ggplot2)

  ggplot(p_Sein) +
    geom_sf(aes(fill = LYSA_cat), color = "grey60", size = 0.2) +
    facet_wrap(~ type_cancer) +
    scale_fill_manual(
      values = lysa_palette,
      drop = FALSE,
      name = "IC95%(SIR_BRUT)"
    ) +
    labs(
      title = "Zones de sur- ou sous-incidence significative selon le SIR_BRUT",
      subtitle = "Cancer du Sein (F) 2010-2017",
      caption = "Bas? sur l'intervalle de confiance ? 95% IC_sup <0.95 & IC_inf >1.05"
    ) +
    coord_sf() +
    theme_void(base_size = 12) +
    theme(
      legend.position = "right",
      plot.title = element_text(face = "bold", size = 14, hjust = 0.5),
      plot.subtitle = element_text(size = 12, hjust = 0.5),
      axis.text = element_blank(),
      axis.ticks = element_blank(),
      axis.title = element_blank(),
      strip.text = element_text(face = "bold", size = 11)
    )

  ##### CARTE SIGNIFICATIVE MELANOME

  library(ggplot2)

  ggplot(p_m?lanome) +
    geom_sf(aes(fill = LYSA_cat), color = "grey60", size = 0.2) +
    facet_wrap(~ type_cancer) +
    scale_fill_manual(
      values = lysa_palette,
      drop = FALSE,
      name = "IC95%(SIR_BRUT)"
    ) +
    labs(
      title = "Zones de sur- ou sous-incidence significative selon le SIR_BRUT",
      subtitle = "Cancer du M?lanome cutan? 2010-2017",
      caption = "Bas? sur l'intervalle de confiance ? 95% IC_sup <0.95 & IC_inf >1.05"
    ) +
    coord_sf() +
    theme_void(base_size = 12) +
    theme(
      legend.position = "right",
      plot.title = element_text(face = "bold", size = 14, hjust = 0.5),
      plot.subtitle = element_text(size = 12, hjust = 0.5),
      axis.text = element_blank(),
      axis.ticks = element_blank(),
      axis.title = element_blank(),
      strip.text = element_text(face = "bold", size = 11)
    )


  #########################################################################################
  nb_cas_total ~ quintileedi2011 + offset(log_expected)

  sir_map_cancer_agr?g? <- sir_map_cancer_agr?g? %>%
    mutate(
      SIR_cat = case_when(
        is.na(SIR_brut)     ~ "donn?es manquantes",
        SIR_brut == 0       ~ "SIR = 0",
        SIR_brut < 0.80     ~ "< 0.80",
        SIR_brut < 0.95     ~ "[0.80 ? 0.95[",
        SIR_brut < 1.05     ~ "[0.95 ? 1.05[",
        SIR_brut < 1.20     ~ "[1.05 ? 1.20[",
        SIR_brut <= 2.00    ~ "[1.20 ? 2.00]",
        SIR_brut > 2.00     ~ "> 2.00"
      ),
      SIR_cat = factor(SIR_cat, levels = names(sir_palette))
    )


 ##### Corriger cette partie demain 08/07/2025

  sir_map_cancer_agr?g? %>%
    dplyr::filter(is.na(SIR_brut))%>%
    dplyr::select(CODE_IRIS,type_cancer,nb_cas_total,cas_attendus)

  sir_map_cancer_agr?g? <- sir_map_cancer_agr?g? %>%
    dplyr::filter(!is.na(SIR_brut))

  sapply(sir_map_cancer_agr?g?,function(x) sum(is.na(x)))

  #### SIR LISSE CANCERS
  library(CARBayes)
  library(sf)
  library(spdep)
  library(dplyr)


  types_cancers <- unique(sir_map_cancer_agr?g?$type_cancer)

  sir_liss?s_par_cancer <- lapply(types_cancers, function(cancer) {
    message("Traitement : ", cancer)

    df_cancer <- sir_map_cancer_agr?g? %>%
      filter(type_cancer == cancer)

    neighbors <- poly2nb(df_cancer)
    W_local   <- nb2mat(neighbors, style = "B", zero.policy = TRUE)

    data_model <- df_cancer %>%
      st_drop_geometry() %>%
      rename(Y = nb_cas_total, E = cas_attendus) %>%
      mutate(region_id = 1:n())
    set.seed(42)
    model <- S.CARbym(
      formula  = Y ~ offset(log(E)),
      family   = "poisson",
      data     = data_model,
      W        = W_local,
      burnin   = 10000,
      n.sample = 50000,
      thin     = 10
    )

    data_model$SIR_liss? <- model$fitted.values / data_model$E
    quantiles <- apply(model$samples$fitted, 2, quantile, probs = c(0.025, 0.975))
    data_model$IC_lisse_inf <- quantiles[1, ] / data_model$E
    data_model$IC_lisse_sup <- quantiles[2, ] / data_model$E
    sir_samples <- model$samples$fitted/data_model$E
    data_model$prob_exces<- colMeans(sir_samples >1.05)
    data_model$type_cancer <- if_else(data_model$prob_exces >0.90,"*","")
    data_model$type_cancer <- cancer

    df_sf <- df_cancer %>% dplyr::select(CODE_IRIS, geometry)

    left_join(data_model, df_sf, by = "CODE_IRIS") %>%
      st_as_sf()
  }) %>% bind_rows()
    sir_palette <- c(
    "SIR = 0"            = "#1a9850",
    "< 0.80"             = "#2166ac",
    "[0.80 ? 0.95["      = "#67a9cf",
    "[0.95 ? 1.05["      = "#f0f0f0",
    "[1.05 ? 1.20["      = "#fdd9a0",
    "[1.20 ? 2.00]"      = "#fdae61",
    "> 2.00"             = "#d73027",
    "donn?es manquantes" = "lightgrey"
  )

  sir_liss?s_par_cancer <- sir_liss?s_par_cancer %>%
    mutate(
      SIR_lisse_cat = case_when(
        is.na(SIR_liss?)     ~ "donn?es manquantes",
        SIR_liss? == 0       ~ "SIR = 0",
        SIR_liss? < 0.80     ~ "< 0.80",
        SIR_liss? < 0.95     ~ "[0.80 ? 0.95[",
        SIR_liss? < 1.05     ~ "[0.95 ? 1.05[",
        SIR_liss? < 1.20     ~ "[1.05 ? 1.20[",
        SIR_liss? <= 2.00    ~ "[1.20 ? 2.00]",
        SIR_liss? > 2.00     ~ "> 2.00"
      ),
      SIR_lisse_cat = factor(SIR_lisse_cat, levels = names(sir_palette)),
      ?toile = if_else(prob_exces > 0.90,"*","")
    )
  library(ggplot2)

  ggplot(sir_liss?s_par_cancer) +
    geom_sf(aes(fill = SIR_lisse_cat), color = "grey70", size = 0.2) +
    geom_sf_text(aes(label = ?toile), size = 3.5, color = "black") +
    scale_fill_manual(
      values = sir_palette,
      name = "SIR liss?",
      na.value = "lightgrey",
      drop = TRUE
    ) +
    facet_wrap(~ type_cancer) +
    labs(
      title = "Carte des SIR_liss? par types de cancers (Mod?le BYM)",
      caption = "* = exc?s significatif (prob_exces > 0.90)"
    ) +
    theme_minimal() +
    theme(
      strip.text = element_text(face = "bold"),
      legend.position = "right",
      axis.text = element_blank(),
      axis.ticks = element_blank(),
      axis.title = element_blank()
    )


 #########################################################

  Bym_poumon <- sir_liss?s_par_cancer %>% filter(type_cancer == "Poumon")
  Bym_Sein <- sir_liss?s_par_cancer %>% filter(type_cancer == "Sein (F)")
  Bym_Prosate  <- sir_liss?s_par_cancer %>% filter(type_cancer == "Prostate (H)")
  Bym_m?lanome <- sir_liss?s_par_cancer %>% filter(type_cancer == "M?lanome cutan?")

  P90_Poumon <- Bym_poumon %>%
    filter(?toile == "*")
  P90_Sein <- Bym_Sein %>%
    filter(?toile == "*")

  P90_Prostate <- Bym_Prosate %>%
    filter(?toile == "*")
  P90_M?lanome <- Bym_m?lanome %>%
    filter(?toile == "*")

  library(writexl)
  write_xlsx(P90_Poumon,"P90_Poumon.xlsx")
  write_xlsx(P90_Sein,"P90_Sein.xlsx")

  write_xlsx(P90_Prostate,"P90_Prostate.xlsx")
  write_xlsx(P90_M?lanome,"P90_M?lanome.xlsx")

  ##### CARTE DES SIR LISSE POUMON

  library(ggplot2)

  ggplot(Bym_poumon) +
    geom_sf(aes(fill = SIR_lisse_cat), color = "grey70", size = 0.2) +
    geom_sf_text(aes(label = ?toile), size = 3.5, color = "black") +
    scale_fill_manual(
      values = sir_palette,
      name = "SIR liss?",
      na.value = "lightgrey",
      drop = TRUE
    ) +
    facet_wrap(~ type_cancer) +
    labs(
      title = "Carte des SIR_liss? Cancer du Poumon,2010-2017",
      caption = "* = exc?s significatif (prob_exces > 0.90)"
    ) +
    theme_void() +
    theme(
      strip.text = element_text(face = "bold"),
      legend.position = "right",
      axis.text = element_blank(),
      axis.ticks = element_blank(),
      axis.title = element_blank()
)

  ##### CARTE DES SIR LISSE MELANOME

  ggplot(Bym_m?lanome) +
    geom_sf(aes(fill = SIR_lisse_cat), color = "grey70", size = 0.2) +
    geom_sf_text(aes(label = ?toile), size = 3.5, color = "black") +
    scale_fill_manual(
      values = sir_palette,
      name = "SIR liss?",
      na.value = "lightgrey",
      drop = TRUE
    ) +
    facet_wrap(~ type_cancer) +
    labs(
      title = "Carte des SIR_liss? Cancer du M?lanome cutan?,2010-2017",
      caption = "* = exc?s significatif (prob_exces > 0.90)"
    ) +
    theme_void() +
    theme(
      strip.text = element_text(face = "bold"),
      legend.position = "right",
      axis.text = element_blank(),
      axis.ticks = element_blank(),
      axis.title = element_blank()
    )

  ##### CARTE DES SIR LISSE SEIN (F)


  ggplot(Bym_Sein) +
    geom_sf(aes(fill = SIR_lisse_cat), color = "grey70", size = 0.2) +
    geom_sf_text(aes(label = ?toile), size = 3.5, color = "black") +
    scale_fill_manual(
      values = sir_palette,
      name = "SIR liss?",
      na.value = "lightgrey",
      drop = TRUE
    ) +
    facet_wrap(~ type_cancer) +
    labs(
      title = "Carte des SIR_liss? Cancer du Sein (F),2010-2017",
      caption = "* = exc?s significatif (prob_exces > 0.90)"
    ) +
    theme_void() +
    theme(
      strip.text = element_text(face = "bold"),
      legend.position = "right",
      axis.text = element_blank(),
      axis.ticks = element_blank(),
      axis.title = element_blank()
    )

  ##### CARTE DES SIR LISSE PROSTATE (H)

  ggplot(Bym_Prosate) +
    geom_sf(aes(fill = SIR_lisse_cat), color = "grey70", size = 0.2) +
    geom_sf_text(aes(label = ?toile), size = 3.5, color = "black") +
    scale_fill_manual(
      values = sir_palette,
      name = "SIR liss?",
      na.value = "lightgrey",
      drop = TRUE
    ) +
    facet_wrap(~ type_cancer) +
    labs(
      title = "Carte des SIR_liss? Cancer de la Prostate (H),2010-2017",
      caption = "* = exc?s significatif (prob_exces > 0.90)"
    ) +
    theme_void() +
    theme(
      strip.text = element_text(face = "bold"),
      legend.position = "right",
      axis.text = element_blank(),
      axis.ticks = element_blank(),
      axis.title = element_blank()
    )
############################################################

  library(tmap)
  tmap_mode("view")

  tm_shape(sir_liss?s_par_cancer) +
    tm_polygons(
      col         = "SIR_lisse_cat",
      palette     = sir_palette,
      title       = "SIR liss? (Bayesien)",
      drop.levels = TRUE,
      textNA      = "Manquant",
      colorNA     = "lightgrey",
      id          = "LIBIRIS",
      popup.vars  = c(
        "Type de cancer" = "type_cancer",
        "Commune"        = "LIBCOM",
        "Nom IRIS"       = "LIBIRIS",
        "SIR liss?"      = "SIR_liss?",
        "IC95 inf."      = "IC_lisse_inf",
        "IC95 sup."      = "IC_lisse_sup",
        "Cas observ?s"   = "Y",
        "Cas attendus"   = "E",
        "* significatif" = "?toile"
      )
    ) +
    tm_text("?toile", size = 1.2, col = "black", just = "center") +
    tm_facets(by = "type_cancer") +
    tm_layout(
      title             = "SIR liss?s par IRIS et par type de cancer",
      legend.outside    = TRUE,
      legend.title.size = 1.1,
      frame             = FALSE
    )


################################################################################

#####################################################################################
#### Mod?lisation liss? par cancer + EDI2011

  library(CARBayes)
  library(dplyr)
  library(sf)
  library(spdep)

  edi2011 <-carte_edi2011_filtr?e %>%
    select(CODE_IRIS,quintileedi2011)

  edi2011 <- st_drop_geometry(edi2011)

  # ? Jointure et pr?-traitement
  df_bym <- sir_liss?s_par_cancer %>%
    left_join(edi2011, by = "CODE_IRIS") %>%
    mutate(
      E = ifelse(E <= 0, 1e-6, E),
      log_E = log(E),
      quintileedi2011 = as.factor(quintileedi2011)
    ) %>%
    st_as_sf()

  #  Initialisation des r?sultats
  results_bym <- list()
  summary_bym <- tibble(
    type_cancer = character(),
    DIC = numeric(),
    WAIC = numeric(),
    LMPL = numeric()
  )

  # ? Boucle sur chaque cancer
  for (cancer in unique(df_bym$type_cancer)) {
    cat("\n? Mod?le BYM pour :", cancer, "\n")

    df_cancer <- df_bym %>%
      filter(type_cancer == cancer) %>%
      filter(!is.na(Y), !is.na(log_E), !is.na(quintileedi2011)) %>%
      distinct(CODE_IRIS, .keep_all = TRUE) %>%
      st_as_sf()

    #  V?rification du volume
    if (nrow(df_cancer) < 10) {
      cat(" Trop peu d'observations pour", cancer, "\n")
      next
    }

    # ? Matrice de voisinage sp?cifique au sous-?chantillon
    nb_cancer <- poly2nb(df_cancer)
    W_cancer <- nb2mat(nb_cancer, style = "B", zero.policy = TRUE)

    # ? Mod?lisation spatiale BYM
    model <- tryCatch(
      S.CARbym(
        formula = Y ~ quintileedi2011 + offset(log_E),
        family = "poisson",
        data = df_cancer,
        W = W_cancer,
        burnin = 10000,
        n.sample = 30000
      ),
      error = function(e) {
        message("X Erreur dans le mod?le pour ", cancer, ": ", e$message)
        return(NULL)
      }
    )

    #  Stockage si le mod?le a r?ussi
    if (!is.null(model) && inherits(model, "CARBayes")) {
      results_bym[[cancer]] <- model

      summary_bym <- summary_bym %>%
        add_row(
          type_cancer = cancer,
          DIC = round(model$modelfit["DIC"], 2),
          WAIC = round(model$modelfit["WAIC"], 2),
          LMPL = round(model$modelfit["LMPL"], 2)
        )
    }
  }



phi <- apply(model$samples$psi, 2, mean)  # moyenne MCMC
df_cancer$phi_bym <- phi

model$summary.results

rr_edi <- exp(model$summary.results["quintileedi20112", "Mean"])

coeffs <- model$summary.results
rr_df <- tibble(
  quintile = rownames(coeffs)[grepl("quintileedi", rownames(coeffs))],
  rr = exp(coeffs[grepl("quintileedi", rownames(coeffs)), "Mean"])
)


plot_rr_bym <- function(model, titre) {
  coeffs <- model$summary.results
  rr_df <- tibble(
    variable = rownames(coeffs)[grepl("quintileedi", rownames(coeffs))],
    rr = exp(coeffs[grepl("quintileedi", rownames(coeffs)), "Mean"]),
    rr_low = exp(coeffs[grepl("quintileedi", rownames(coeffs)), "2.5%"]),
    rr_high = exp(coeffs[grepl("quintileedi", rownames(coeffs)), "97.5%"])
  )

  ggplot(rr_df, aes(x = variable, y = rr)) +
    geom_point(size = 3) +
    geom_errorbar(aes(ymin = rr_low, ymax = rr_high), width = 0.2) +
    geom_hline(yintercept = 1, linetype = "dashed", color = "red") +
    labs(title = titre, x = "Quintile EDI", y = "Risque relatif ajust? (RR)") +
    theme_minimal()
}

plot_rr_bym(results_bym[["Poumon"]], "Effets EDI ? Cancer du Poumon (BYM)")




library(ggplot2)
library(dplyr)
library(patchwork)  # pour combiner les graphiques

#  Fonction pour tracer les RR par cancer
plot_rr_bym <- function(model, cancer_name) {
  coeffs <- model$summary.results

  rr_df <- tibble(
    quintile = rownames(coeffs)[grepl("quintileedi", rownames(coeffs))],
    rr = exp(coeffs[grepl("quintileedi", rownames(coeffs)), "Mean"]),
    rr_low = exp(coeffs[grepl("quintileedi", rownames(coeffs)), "2.5%"]),
    rr_high = exp(coeffs[grepl("quintileedi", rownames(coeffs)), "97.5%"])
  )%>%
    mutate(
      quintile = gsub("quintileedi2011","",quintile),
      quintile = gsub("quintileedi","",quintile),
      quintile = as.factor(quintile)
    )

  ggplot(rr_df, aes(x = quintile, y = rr)) +
    geom_point(size = 3, color = "#0072B2") +
    geom_errorbar(aes(ymin = rr_low, ymax = rr_high), width = 0.2) +
    geom_hline(yintercept = 1, linetype = "dashed", color = "gray") +
    labs(
      title = paste("RR par quintile EDI2011", cancer_name),
      x = "Quintile EDI2011",
      y = "Risque relatif (RR) BYM"
    ) +
    theme_minimal()
}

# ? G?n?ration des 4 graphiques
p_melanome <- plot_rr_bym(results_bym[["M?lanome cutan?"]], "M?lanome cutan?")
p_poumon   <- plot_rr_bym(results_bym[["Poumon"]], "Poumon")
p_prostate <- plot_rr_bym(results_bym[["Prostate (H)"]], "Prostate (H)")
p_sein     <- plot_rr_bym(results_bym[["Sein (F)"]], "Sein (F)")

# ?? Combinaison des 4 graphiques
(p_melanome | p_poumon) /
  (p_prostate | p_sein)

p_sein

####### CARTE DES RR PAR CANCERS-EDI

library(sp)
library(spdep)
library(CARBayes)
library(dplyr)
library(sf)
library(ggplot2)
library(patchwork)

#? Pr?paration des donn?es
df_base <- sir_liss?s_par_cancer %>%
  left_join(edi2011, by = "CODE_IRIS") %>%
  filter(!is.na(Y), !is.na(E), !is.na(quintileedi2011)) %>%
  mutate(
    log_E = log(ifelse(E <= 0, 1e-6, E)),  # ?viter log(0)
    quintileedi2011 = as.factor(quintileedi2011)
  )


#  Initialisation
results_bym <- list()
rr_maps <- list()
rr_boxes <- list()

#  Boucle par type de cancer
for (cancer in unique(df_base$type_cancer)) {
  cat("\n? Mod?le BYM pour :", cancer, "\n")

  df_cancer <- df_base %>% filter(type_cancer == cancer)

  spatial_data <- as(df_cancer, "Spatial")

  nb <- poly2nb(spatial_data)
  W <- nb2mat(nb, style = "B", zero.policy = TRUE)

  model <- S.CARbym(
    formula = Y ~ quintileedi2011 + offset(log_E),
    family = "poisson",
    data = spatial_data,
    W = W,
    burnin = 20000,
    n.sample = 100000,
    thin = 10
  )

  results_bym[[cancer]] <- model

  #  RR ajust?
  spatial_data$RR_bym <- model$fitted.values / spatial_data$E
  lambda_samples  <- model$samples$fitted
  sir_samples  <- sweep(lambda_samples,2,spatial_data$E,FUN = "/")
  prob_exces <- colMeans(sir_samples > 1.05)
  spatial_data$?toile <- ifelse(prob_exces > 0.90,"*","")

  # Cat?gorisation
  spatial_data$RR_cat <- cut(spatial_data$RR_bym,
                             breaks = c(0, 0.8, 0.95, 1.05, 1.2, 2, Inf),
                             labels = c("< 0.80", "[0.80 ? 0.95[", "[0.95 ? 1.05[", "[1.05 ? 1.20[", "[1.20 ? 2.00]", "> 2.00"),
                             include.lowest = TRUE)

  spatial_sf <- st_as_sf(spatial_data)



  #  Carte RR
  rr_maps[[cancer]] <- ggplot(spatial_sf) +
    geom_sf(aes(fill = RR_cat), color = "grey60", size = 0.1) +
    geom_sf_text(aes(label = ?toile),size = 4,color = "black")+
    scale_fill_manual(
      values = c(
        "< 0.80" = "#2166ac",
        "[0.80 ? 0.95[" = "#c6dbef",
        "[0.95 ? 1.05[" = "#FFFFFF",
        "[1.05 ? 1.20[" = "#fdd9a0",
        "[1.20 ? 2.00]"= "#fdae61",
        "> 2.00" = "#d73027"
      ),
      name = "RR BYM"
    ) +
    labs(title = paste("RR liss? BYM ?", cancer),
         caption = "* P(RR>1.05)>0.90"
    ) +

    theme_void()

}


   #  Affichage des cartes uniquement
rr_maps[["Sein (F)"]] +
  rr_maps[["Poumon"]] +
  rr_maps[["Prostate (H)"]] +
  rr_maps[["M?lanome cutan?"]]


(rr_maps[["Sein (F)"]] | rr_maps[["Poumon"]]) /
  (rr_maps[["Prostate (H)"]] | rr_maps[["M?lanome cutan?"]])


rr_maps[["Sein (F)"]]

rr_maps[["Poumon"]]

rr_maps[["Prostate (H)"]]
rr_maps[["M?lanome cutan?"]]





library(ggplot2)

ggplot(spatial_sf, aes(x = factor(quintileedi2011), y = RR_bym)) +
  geom_boxplot(fill = "#de2d26", color = "black", outlier.color = "gray30") +
  facet_wrap(~ type_cancer )+
  labs(
    title = "Distribution des risques relatifs BYM selon le quintile EDI Tous-cancers(2010-2017)",
    x = "Quintile EDI (1 = favoris?, 5 = d?favoris?)",
    y = "RR ajust? (i/Ei)"
  ) +
  theme_minimal()


###Carte RR SEIN (F)

rr_sein <- spatial_sf %>%
  filter(type_cancer == "Sein (F)")

library(ggplot2)

ggplot(rr_sein, aes(x = factor(quintileedi2011), y = RR_bym)) +
  geom_boxplot(fill = "#de2d26", color = "black", outlier.color = "gray30") +
  labs(
    title = "Distribution des risques relatifs BYM selon le quintile EDI-Sein(F)",
    x = "Quintile EDI (1 = favoris?, 5 = d?favoris?)",
    y = "RR ajust? (??_i/Ei)"
  ) +
  theme_minimal()




rr_Prostate <- spatial_sf %>%
  filter(type_cancer == "Prostate(H)")
rr_sein <- spatial_sf %>%
  filter(type_cancer == "Sein (F)")
rr_sein <- spatial_sf %>%
  filter(type_cancer == "Sein (F)")


model_bym
rr
library(writexl)
write_xlsx(rr,"rr.xlsx")
write_xlsx(zone_sans_cas,"Zone avec 0 cas.xlsx")
write_xlsx(zone_liss?,"zone liss? p90.xlsx")

################################################################################
